## Load necessary packages

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset, ConcatDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.utils import weight_norm

In [2]:
torch.cuda.is_available()

False

In [3]:
import sys
sys.path.append('/Users/AFischer/PycharmProjects/cocoon-project')
sys.path.append('/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database')

In [4]:
import math
import wfdb
import numpy as np
import pandas as pd 
import constants as c
from src_pre_term_database.load_dataset import build_signal_dataframe, build_clinical_information_dataframe, build_demographics_dataframe
from src_pre_term_database.modeling import SaveBestModel, LSTMModel, LSTMCombinedModel, LSTMStateful, LSTMStatefulClassificationOriginalSequence, LSTMStatefulClassificationFeatureSequence, TCN, get_num_levels_based_on_receptive_field
from src_pre_term_database.optimization import ObjectiveTcnFeatureModel, OptimizationTCNFeatureSequence, OptimizationLSTM, OptimizationCombinedLSTM, OptimizationStatefulLSTM, ObjectiveLSTMFeatureCombinedModel
from src_pre_term_database.final_train import final_train_tcn_feature_sequence, final_train_lstm_feature_sequence
from src_pre_term_database.evaluation import evaluate_tcn_feature_sequence, evaluate_lstm_feature_sequence, create_interval_matrix
from utils import read_settings, replace_inf_with_zero, replace_na_with_zero, get_correlated_columns, remove_correlated_columns
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.signal import butter, filtfilt
from src_pre_term_database.visualizing import plot_histogram, plot_distribution_feature
from src_pre_term_database.data_processing_and_feature_engineering import fit_and_transform_data, create_filtered_channels, remove_first_n_samples_of_signals, remove_last_n_samples_of_signals, train_val_test_split, feature_label_split, pad_sequences_to_fixed_length, pad_sequences_to_fixed_length_df, custom_sort_for_stateful_lstm, NonOverlappingSequencesDataset, sort_in_descending_order_of_occurrence_count, custom_sort_seq_lengths, compute_non_padded_timesteps_per_batch_of_original_seq, get_feature_names, StaticDataset, CombinedDataset, ColumnDropperTransformer, preprocess_static_data, create_dataloader_if_batch_sizes_unequal, create_dataloader_if_batch_sizes_equal, create_tensordataset_reduced_seq_length, generate_custom_data_loaders, calculate_feature_over_fixed_time_window, custom_calculate_feature_over_fixed_time_window, custom_calculate_feature_over_splited_fixed_time_window, generate_feature_data_loaders
from pandas._testing import assert_frame_equal
from IPython.display import Latex
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, roc_auc_score, average_precision_score
from sklearn.impute import SimpleImputer
import datetime
import csv
from timeit import default_timer as timer
import bayes_opt
import antropy as ant
import optuna
import pickle
import time
import joblib
import random
from typing import List
import wandb
plt.style.use('ggplot')

In [5]:
settings_path = '/Users/AFischer/PycharmProjects/cocoon-project/references/settings'
file_paths = read_settings(settings_path, 'file_paths')

SIGNAL_COLUMN_NAMES = ['1', '1_DOCFILT-4-0.08-4', '1_DOCFILT-4-0.3-3', '1_DOCFILT-4-0.3-4',
                       '2', '2_DOCFILT-4-0.08-4', '2_DOCFILT-4-0.3-3', '2_DOCFILT-4-0.3-4',
                       '3', '3_DOCFILT-4-0.08-4', '3_DOCFILT-4-0.3-3', '3_DOCFILT-4-0.3-4']

In [6]:
data_path = file_paths['data_path']

## Load in data

In [8]:
df_signals = build_signal_dataframe(data_path, settings_path)

  0%|          | 0/300 [00:00<?, ?it/s]

In [9]:
df_signals.head()

,rec_id,channel_1,channel_1_filt_0.08_4_hz,channel_1_filt_0.3_3_hz,channel_1_filt_0.3_4_hz,channel_2,channel_2_filt_0.08_4_hz,channel_2_filt_0.3_3_hz,channel_2_filt_0.3_4_hz,channel_3,channel_3_filt_0.08_4_hz,channel_3_filt_0.3_3_hz,channel_3_filt_0.3_4_hz
0,1391,-0.404669,-0.010681,-0.003128,-0.008392,-0.284733,-0.007401,-0.002213,-0.006027,0.555428,0.014954,0.004349,0.011597
1,1391,-0.407111,-0.061646,-0.020523,-0.048142,-0.283513,-0.042725,-0.014725,-0.034409,0.555428,0.086290,0.028305,0.066377
2,1391,-0.401007,-0.157244,-0.061494,-0.120928,-0.281071,-0.108949,-0.044099,-0.086595,0.549325,0.220035,0.084764,0.166629
3,1391,-0.394903,-0.239490,-0.115053,-0.179065,-0.276188,-0.165866,-0.082399,-0.128328,0.545663,0.335088,0.158618,0.246662
4,1391,-0.396124,-0.255055,-0.154498,-0.181201,-0.277409,-0.176089,-0.110399,-0.129778,0.537118,0.356832,0.213169,0.249561


A Butterworth filter with forward and backward filter is used to filter the signals. The combined filter has zero phase. Different bandwidths can be used.

In [10]:
df_signals_new = create_filtered_channels(df_signals, ['channel_1', 'channel_2', 'channel_3'], 
                                          [[0.34, 1], [0.08, 4], [0.3, 3], [0.3, 4]], fs=20, order=4)

In [11]:
df_signals_new.head()

,rec_id,channel_1,channel_1_filt_0.34_1_hz,channel_1_filt_0.08_4_hz,channel_1_filt_0.3_3_hz,channel_1_filt_0.3_4_hz,channel_2,channel_2_filt_0.34_1_hz,channel_2_filt_0.08_4_hz,channel_2_filt_0.3_3_hz,channel_2_filt_0.3_4_hz,channel_3,channel_3_filt_0.34_1_hz,channel_3_filt_0.08_4_hz,channel_3_filt_0.3_3_hz,channel_3_filt_0.3_4_hz
0,1391,-0.404669,0.001757,-0.010257,0.002399,0.002381,-0.284733,0.000609,-0.005152,0.000943,0.000952,0.555428,-0.002105,0.013392,-0.003485,-0.003449
1,1391,-0.407111,0.001033,-0.011189,0.001060,0.000447,-0.283513,0.000533,-0.004269,0.000500,0.000779,0.555428,-0.001343,0.014355,-0.001834,-0.000976
2,1391,-0.401007,0.000331,-0.010475,0.000440,0.000108,-0.281071,0.000471,-0.003259,0.000366,0.000696,0.549325,-0.000594,0.013447,-0.000928,-0.000350
3,1391,-0.394903,-0.000324,-0.008425,0.000770,0.001094,-0.276188,0.000428,-0.002107,0.000722,0.000741,0.545663,0.000129,0.010921,-0.000996,-0.001336
4,1391,-0.396124,-0.000908,-0.006563,0.001628,0.001937,-0.277409,0.000409,-0.000709,0.001503,0.001070,0.537118,0.000810,0.008568,-0.001605,-0.002184


In [12]:
del df_signals

The first and last 180 seconds of the signals should be ignored since these intervals contain transient effects of the filters(source: https://physionet.org/content/tpehgdb/1.0.1/#ref1).

The sampling frequency is 20 Hz, so the first and last 20*180 datapoints should be removed.

In [13]:
df_signals_new = remove_first_n_samples_of_signals(df_signals_new, n=3600)

The number of data points before removing the first 3600 data points (per rec_id) is: 10655901
The number of data points after removing the first 3600 data points (per rec_id) is: 9575901


In [14]:
df_signals_new = remove_last_n_samples_of_signals(df_signals_new, n=3600)

The number of data points before removing the last 3600 data points (per rec_id) is: 9575901
The number of data points after removing the last 3600 data points (per rec_id) is: 8495901


In [9]:
data_path

'/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/term-preterm-ehg-database-1.0.1/term-preterm-ehg-database-1.0.1'

In [16]:
df_signals_new.to_csv(f'{data_path}/df_signals_filt.csv', sep=';')

We need to check the distribution of the sequence length of all the rec ids. Later on, we will need to pad/truncate every sequence to the same length. Based on the plot we will choose 28800 (which is the mode) as the fixed sequence length we will pad/truncate all the sequences to.

In [30]:
plot_distribution_feature(df_signals_new.groupby('rec_id', sort=False).agg({'rec_id': 'size'}).rename(columns={'rec_id': 'count'}).reset_index(), 'count', categorical=False)

df_clinical_information contains the labels (the column premature) for each rec_id. We will add this column to the signals dataframe. We want to build a model to predict premature

In [7]:
df_clinical_information = build_clinical_information_dataframe(data_path, settings_path)

In [8]:
df_clinical_information.head()

,rec_id,gestation,gestation_at_rec_time,group,premature,early
0,1007,35.00,31.29,>=26-PRE,t,f
1,1021,38.57,22.29,<26-TERM,f,t
2,1022,38.57,31.00,>=26-TERM,f,f
3,1027,37.14,31.29,>=26-TERM,f,f
4,1029,38.57,31.00,>=26-TERM,f,f


In [33]:
df_signals = df_signals_new.merge(df_clinical_information[['rec_id', 'premature']], how='left', on='rec_id')

In [34]:
# Assign 0 to non-premature cases and assign 1 to premature cases
lb = LabelEncoder()
df_signals['premature'] = lb.fit_transform(df_signals['premature'])

We use df_clinical_information for this operation as this df is much smaller and therefore way faster. Also the
order of the dataframe does not matter as each rec_id occurs only once in the data. The train/val/test split is made with stratification

In [9]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_clinical_information, 'premature', 
                                                                      test_ratio=0.2, 
                                                                      shuffle=True, random_state=0)

In [36]:
X_train['rec_id'].nunique()

180

In [37]:
X_val['rec_id'].nunique()

60

In [38]:
X_test['rec_id'].nunique()

60

In [94]:
X_test

,rec_id,gestation,gestation_at_rec_time,group,early
0,737,40.00,32.00,>=26-TERM,f
1,1183,41.14,31.14,>=26-TERM,f
2,1537,40.43,22.57,<26-TERM,t
3,827,39.86,24.00,<26-TERM,t
4,916,39.29,30.86,>=26-TERM,f
5,1335,40.43,30.57,>=26-TERM,f
6,753,40.14,31.86,>=26-TERM,f
7,930,41.29,25.29,<26-TERM,t
8,625,38.71,31.86,>=26-TERM,f
9,697,39.43,22.86,<26-TERM,t


In [95]:
X_test['time_between_rec_and_birth'] = X_test['gestation']-X_test['gestation_at_rec_time']

In [99]:
X_test[X_test['time_between_rec_and_birth']<=6]

,rec_id,gestation,gestation_at_rec_time,group,early,time_between_rec_and_birth
10,858,27.71,23.86,<26-PRE,t,3.85
14,1532,36.71,31.57,>=26-PRE,f,5.14
33,553,37.00,31.00,>=26-TERM,f,6.00
39,1007,35.00,31.29,>=26-PRE,f,3.71
48,914,34.29,30.71,>=26-PRE,f,3.58


# Static data preprocessing

In [10]:
df_demographics = build_demographics_dataframe(data_path, settings_path)

  0%|          | 0/300 [00:00<?, ?it/s]

In [11]:
df_static_information = df_demographics.merge(df_clinical_information, how='left', on=c.REC_ID_NAME)

In [12]:
df_static_information.head()

,rec_id,age,parity,abortions,weight,hypertension,diabetes,placental_position,bleeding_first_trimester,bleeding_second_trimester,funneling,smoker,gestation,gestation_at_rec_time,group,premature,early
0,1391,25.0,0.0,0.0,65.0,no,no,front,no,no,negative,no,35.57,24.43,<26-PRE,t,t
1,572,27.0,0.0,0.0,59.0,no,no,end,no,no,negative,no,40.00,32.29,>=26-TERM,f,f
2,1346,41.0,2.0,0.0,75.0,no,yes,front,no,no,negative,no,37.29,22.14,<26-TERM,f,t
3,1360,NaN,NaN,NaN,46.0,None,None,end,no,no,negative,None,41.14,22.86,<26-TERM,f,t
4,583,24.0,NaN,NaN,105.0,None,None,front,no,no,negative,None,40.43,31.00,>=26-TERM,f,f


In [13]:
features_to_use_static = ['hypertension', 'diabetes', 'placental_position', 'bleeding_first_trimester',
                          'bleeding_second_trimester', 'funneling', 'smoker', 'parity', 'abortions', 
                          'age', 'weight', 'gestation_at_rec_time']

In [14]:
X_train_arr_static, y_train_arr_static, selected_columns, _, rec_id_list_train = preprocess_static_data(df_static_information, 
                                                                                                        X_train, 
                                                                                                        X_train, 
                                                                                                        threshold_correlation=0.85)
X_val_arr_static, y_val_arr_static, _, _, rec_id_list_val = preprocess_static_data(df_static_information, 
                                                                                   X_train, X_val, 
                                                                                   threshold_correlation=0.85)
X_test_arr_static, y_test_arr_static, _, _, rec_id_list_test = preprocess_static_data(df_static_information, 
                                                                                      X_train, X_test, 
                                                                                      threshold_correlation=0.85)


## First try a simple Logistic Regresion model or GBM as a quick check to see if the static features have predictive power

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression(penalty= 'l2', C=1, class_weight={0:1, 1:6.83})
logreg.fit(X_train_arr_static, y_train_arr_static)

LogisticRegression(C=1, class_weight={0: 1, 1: 6.83})

In [17]:
y_pred = logreg.predict(X_test_arr_static)

In [18]:
y_pred

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1])

In [19]:
y_test_arr_static

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
logreg.predict_proba(X_test_arr_static)

array([[0.30714166, 0.69285834],
       [0.23662688, 0.76337312],
       [0.56391401, 0.43608599],
       [0.67808722, 0.32191278],
       [0.72042904, 0.27957096],
       [0.43711796, 0.56288204],
       [0.83693709, 0.16306291],
       [0.39759549, 0.60240451],
       [0.85363541, 0.14636459],
       [0.42929876, 0.57070124],
       [0.25835824, 0.74164176],
       [0.71383685, 0.28616315],
       [0.19350997, 0.80649003],
       [0.57391834, 0.42608166],
       [0.59481192, 0.40518808],
       [0.76884315, 0.23115685],
       [0.17768783, 0.82231217],
       [0.6689457 , 0.3310543 ],
       [0.27735986, 0.72264014],
       [0.6725925 , 0.3274075 ],
       [0.27541435, 0.72458565],
       [0.24158865, 0.75841135],
       [0.22694321, 0.77305679],
       [0.54765411, 0.45234589],
       [0.11191299, 0.88808701],
       [0.72712329, 0.27287671],
       [0.43700359, 0.56299641],
       [0.34199758, 0.65800242],
       [0.68880002, 0.31119998],
       [0.43721937, 0.56278063],
       [0.

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test_arr_static, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.50      0.63        52
           1       0.13      0.50      0.21         8

    accuracy                           0.50        60
   macro avg       0.50      0.50      0.42        60
weighted avg       0.77      0.50      0.58        60



In [22]:
print(f'Precision score: {precision_score(y_test_arr_static, y_pred)}')
print(f'Recall score: {recall_score(y_test_arr_static, y_pred)}')
print(f'F1 score: {f1_score(y_test_arr_static, y_pred)}')
print(f'Average precision score: {average_precision_score(y_test_arr_static, logreg.predict_proba(X_test_arr_static)[:,1])}')
print(f'AUC score: {roc_auc_score(y_test_arr_static, logreg.predict_proba(X_test_arr_static)[:,1])}')

Precision score: 0.13333333333333333
Recall score: 0.5
F1 score: 0.2105263157894737
Average precision score: 0.11530431524019763
AUC score: 0.38221153846153844


In [52]:
# fit the model on the whole dataset
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=1000)
model.fit(X_train_arr_static, y_train_arr_static)
y_pred_gbm = model.predict(X_test_arr_static)

In [53]:
model.predict_proba(X_test_arr_static)

array([[9.99999974e-01, 2.58236974e-08],
       [5.22041370e-06, 9.99994780e-01],
       [9.98011116e-01, 1.98888357e-03],
       [9.99996996e-01, 3.00438298e-06],
       [9.99992909e-01, 7.09075707e-06],
       [9.98368809e-01, 1.63119056e-03],
       [9.99999791e-01, 2.08823440e-07],
       [9.99756395e-01, 2.43605233e-04],
       [1.00000000e+00, 4.21907230e-10],
       [9.99970784e-01, 2.92163198e-05],
       [9.99996173e-01, 3.82737390e-06],
       [9.99998960e-01, 1.04033319e-06],
       [9.99862915e-01, 1.37084623e-04],
       [1.00000000e+00, 9.79896982e-11],
       [9.99994348e-01, 5.65151796e-06],
       [9.99999998e-01, 2.31716253e-09],
       [9.99999987e-01, 1.30114727e-08],
       [9.58770075e-01, 4.12299251e-02],
       [9.99975655e-01, 2.43452461e-05],
       [9.99999273e-01, 7.27189794e-07],
       [1.00000000e+00, 1.74009620e-10],
       [1.68589484e-01, 8.31410516e-01],
       [3.13348122e-06, 9.99996867e-01],
       [9.99994667e-01, 5.33303796e-06],
       [2.053447

In [54]:
print(f'Precision score: {precision_score(y_test_arr_static, y_pred_gbm)}')
print(f'Recall score: {recall_score(y_test_arr_static, y_pred_gbm)}')
print(f'F1 score: {f1_score(y_test_arr_static, y_pred_gbm)}')
print(f'Average precision score: {average_precision_score(y_test_arr_static, model.predict_proba(X_test_arr_static)[:,1])}')
print(f'AUC score: {roc_auc_score(y_test_arr_static, model.predict_proba(X_test_arr_static)[:,1])}')

Precision score: 0.14285714285714285
Recall score: 0.125
F1 score: 0.13333333333333333
Average precision score: 0.16871865724833768
AUC score: 0.5576923076923077


## Conclusion: Predictive power of the static features seems to be quite limited. No hyperparameter optimization done however

## Hyperparameter search for TCN and LSTM feature sequence model

In [9]:
# EHG data to use for modelling
features_to_use = ['channel_1_filt_0.34_1_hz', 'channel_2_filt_0.34_1_hz', 'channel_3_filt_0.34_1_hz']
# Either 'lstm' or 'tcn'
model_name = 'lstm'
# Either 'sample_entropy', 'peak_frequency', 'median_frequency'
feature_name = 'peak_frequency'
# Either True or False -> Add static clinical data for modelling
add_static_data = True

In [10]:
# df_signals = build_signal_dataframe(data_path, settings_path)
# df_signals_new = create_filtered_channels(df_signals, ['channel_1', 'channel_2', 'channel_3'],
#                                           [[0.34, 1], [0.08, 4], [0.3, 3], [0.3, 4]], fs=20, order=4)
# df_signals_new = remove_first_n_samples_of_signals(df_signals_new, n=3600)
# df_signals_new = remove_last_n_samples_of_signals(df_signals_new, n=3600)

df_signals_new = pd.read_csv(f'{data_path}/df_signals_filt.csv', sep=';')

df_clinical_information = build_clinical_information_dataframe(data_path, settings_path)
df_signals = df_signals_new.merge(df_clinical_information[[c.REC_ID_NAME, 'premature']],
                                  how='left', on=c.REC_ID_NAME)

# Assign 0 to non-premature cases and assign 1 to premature cases
lb = LabelEncoder()
df_signals['premature'] = lb.fit_transform(df_signals['premature'])

X_train, X_val, X_test, y_train, _, _ = train_val_test_split(df_clinical_information, 'premature', test_ratio=0.2,
                                                             shuffle=True, random_state=0)

df_demographics = build_demographics_dataframe(data_path, settings_path)
df_static_information = df_demographics.merge(df_clinical_information, how='left', on=c.REC_ID_NAME)

current_date_and_time = "{:%Y-%m-%d_%H-%M}".format(datetime.datetime.now())

# Count the number of static features
if add_static_data:
    _, _, selected_columns_fit_static, _, _ = preprocess_static_data(df_static_information, X_train, X_train,
                                                                     threshold_correlation=0.85)

    num_static_features = len(selected_columns_fit_static)
    out_file = f'/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model/hyper_parameter_opt/{model_name}_data_trials_feature_{feature_name}_combined_{current_date_and_time}.csv'

if not add_static_data:
    num_static_features = 0
    out_file = f'/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model/hyper_parameter_opt/{model_name}_data_trials_feature_{feature_name}_{current_date_and_time}.csv'

# File to save results
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'train_time'])
of_connection.close()

out_path_model = '/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model'

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())

if model_name == 'tcn':
    objective = ObjectiveTcnFeatureModel(df_signals, df_clinical_information, df_static_information, X_train, X_val,
                                         feature_name=feature_name, features_to_use=features_to_use,
                                         out_file=out_file, add_static_data=add_static_data,
                                         num_static_features=num_static_features)

elif model_name == 'lstm' and not add_static_data:
    objective = ObjectiveLSTMFeatureModel(df_signals, df_clinical_information, df_static_information,
                                          X_train, X_val,
                                          feature_name=feature_name, features_to_use=features_to_use,
                                          out_file=out_file, add_static_data=add_static_data)

elif model_name == 'lstm' and add_static_data:
    objective = ObjectiveLSTMFeatureCombinedModel(df_signals, df_clinical_information, df_static_information,
                                                  X_train, X_val, feature_name=feature_name,
                                                  features_to_use=features_to_use, add_static_data=add_static_data,
                                                  num_static_features=num_static_features, out_file=out_file)

study.optimize(objective, n_trials=50)
print(study.best_trial)

if add_static_data:
    joblib.dump(study,
                f'{out_path_model}/hyper_opt_{model_name}_feature_{feature_name}_combined_{current_date_and_time}.pkl')

if not add_static_data:
    joblib.dump(study,
                f'{out_path_model}/hyper_opt_{model_name}_feature_{feature_name}_{current_date_and_time}.pkl')

  0%|          | 0/300 [00:00<?, ?it/s]

The number of data points before removing the first 3600 data points (per rec_id) is: 10655901
The number of data points after removing the first 3600 data points (per rec_id) is: 9575901
The number of data points before removing the last 3600 data points (per rec_id) is: 9575901
The number of data points after removing the last 3600 data points (per rec_id) is: 8495901


  0%|          | 0/300 [00:00<?, ?it/s]

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer simple_imp_median (type SimpleImputer) does not provide get_feature_names. Will return input column names if available

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer standard_scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer simple_imp_mean (type SimpleImputer) does not provide get_feature_names. Will return input column names if available

[I 2022-05-10 07:32:02,224] A new study created in memory with name: no-name-62915608-4231-42a6-9cfa-78634d5f01ac


Params: {'learning_rate': 0.0028232460509612016, 'layer_dim': 3, 'hidden_dim_seq': 13, 'hidden_dim_static': 24, 'weight_decay': 0.00024827927129509274, 'bidirectional': False, 'num_epochs': 10, 'batch_size': 40, 'feature_name': 'peak_frequency', 'drop_out_lstm': 0.4681939135551061}


...

The hidden states and cells are resetted at the beginning of epoch 3, batch 0 of train_loader
The hidden states and cells are resetted at the beginning of epoch 3, batch 5 of train_loader
The hidden states and cells are resetted at the beginning of epoch 3, batch 10 of train_loader
The hidden states and cells are resetted at the beginning of epoch 3, batch 0 of val_loader
Epoch [3/10] Training loss: 1.2526	 Validation loss: 1.1872
--------------------------------------------------
EarlyStopping counter: 1 out of 2


/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer simple_imp_median (type SimpleImputer) does not provide get_feature_names. Will return input column names if available

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer standard_scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer simple_imp_mean (type SimpleImputer) does not provide get_feature_names. Will return input column names if available

/Users/AFischer/PycharmProjects/cocoon-project/src_pre_term_database/data_processing_and_feature_engineering.py:211: UserWarning:

Transformer simple_imp_median (type SimpleImputer) does not provide get_feature_nam

The hidden states and cells are resetted at the beginning of epoch 4, batch 0 of train_loader
The hidden states and cells are resetted at the beginning of epoch 4, batch 5 of train_loader
The hidden states and cells are resetted at the beginning of epoch 4, batch 10 of train_loader


[I 2022-05-10 09:10:48,619] Trial 49 finished with value: 1.223467469215393 and parameters: {'learning_rate': 5.103949361486742e-05, 'layer_dim': 1, 'hidden_dim_seq': 9, 'hidden_dim_static': 22, 'weight_decay': 0.00024991933898231577, 'bidirectional': False, 'batch_size': 60, 'activation_or_batchnorm_layer': None, 'n_linear_layers': 1}. Best is trial 44 with value: 1.1677402089039484.


The hidden states and cells are resetted at the beginning of epoch 4, batch 0 of val_loader
Epoch [4/10] Training loss: 1.2426	 Validation loss: 1.2235
--------------------------------------------------
EarlyStopping counter: 2 out of 2
Early stopping
TRAINING COMPLETE
FrozenTrial(number=44, values=[1.1677402089039484], datetime_start=datetime.datetime(2022, 5, 10, 9, 4, 1, 158725), datetime_complete=datetime.datetime(2022, 5, 10, 9, 4, 53, 370875), params={'learning_rate': 3.3402932379737195e-05, 'layer_dim': 1, 'hidden_dim_seq': 6, 'hidden_dim_static': 20, 'weight_decay': 0.02455295385601926, 'bidirectional': False, 'batch_size': 10, 'activation_or_batchnorm_layer': None, 'n_linear_layers': 1}, distributions={'learning_rate': LogUniformDistribution(high=0.1, low=1e-05), 'layer_dim': IntUniformDistribution(high=3, low=1, step=1), 'hidden_dim_seq': IntUniformDistribution(high=15, low=5, step=1), 'hidden_dim_static': IntUniformDistribution(high=25, low=15, step=1), 'weight_decay': LogUn

## Final train of tcn and lstm feature sequence model

In [39]:
out_path_model = '/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model'

In [25]:
# Model name is either 'tcn_feature_sequence' or 'lstm_feature_sequence'
model_name = 'tcn_feature_sequence'
# Name of the file where the results of the hyperparameter search is saved
optimal_params_file_name = 'tcn_data_trials_feature_median_frequency_combined_copies_2022-06-08_07-58.csv'

path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{optimal_params_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')

In [26]:
hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0]

"{'learning_rate': 0.00035250661010214837, 'num_hidden_units_per_layer': 33, 'weight_decay': 0.00037363067277954937, 'kernel_size': 5, 'num_epochs': 10, 'drop_out': 0.2512052057078852, 'batch_size': 40, 'feature_name': 'median_frequency', 'num_levels': 1, 'stride': 1, 'optional_model': Sequential(\n  (0): ReLU()\n  (1): Linear(in_features=33, out_features=12, bias=True)\n  (2): ReLU()\n)}"

In [27]:
# eval does not work for the case of LSTM with static data, so you need to hard copy the best params for these
# cases
#best_params = eval(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])


best_params = {'learning_rate': 0.00035250661010214837, 'num_hidden_units_per_layer': 33, 
               'weight_decay': 0.00037363067277954937, 'kernel_size': 5, 'num_epochs': 10, 
               'drop_out': 0.2512052057078852, 'batch_size': 40, 'feature_name': 'median_frequency', 
               'num_levels': 1, 'stride': 1, 
               'optional_model': nn.Sequential(nn.ReLU(), nn.Linear(in_features=33, 
                                                                    out_features=12, bias=True), nn.ReLU())}

print(f'best params: {best_params}')

features_to_use = ['channel_1_filt_0.34_1_hz', 'channel_2_filt_0.34_1_hz', 'channel_3_filt_0.34_1_hz']
add_static_data = True
copies = True

best params: {'learning_rate': 0.00035250661010214837, 'num_hidden_units_per_layer': 33, 'weight_decay': 0.00037363067277954937, 'kernel_size': 5, 'num_epochs': 10, 'drop_out': 0.2512052057078852, 'batch_size': 40, 'feature_name': 'median_frequency', 'num_levels': 1, 'stride': 1, 'optional_model': Sequential(
  (0): ReLU()
  (1): Linear(in_features=33, out_features=12, bias=True)
  (2): ReLU()
)}


In [28]:
# df_signals = build_signal_dataframe(data_path, settings_path)
# df_signals_new = create_filtered_channels(df_signals, ['channel_1', 'channel_2', 'channel_3'],
#                                           [[0.34, 1], [0.08, 4], [0.3, 3], [0.3, 4]], fs=20, order=4)
# df_signals_new = remove_first_n_samples_of_signals(df_signals_new, n=3600)
# df_signals_new = remove_last_n_samples_of_signals(df_signals_new, n=3600)

df_signals_new = pd.read_csv(f'{data_path}/df_signals_filt.csv', sep=';')

df_clinical_information = build_clinical_information_dataframe(data_path, settings_path)
df_signals = df_signals_new.merge(df_clinical_information[[c.REC_ID_NAME, 'premature']],
                                  how='left', on=c.REC_ID_NAME)

df_demographics = build_demographics_dataframe(data_path, settings_path)
df_static_information = df_demographics.merge(df_clinical_information, how='left', on=c.REC_ID_NAME)

# Assign 0 to non-premature cases and assign 1 to premature cases
lb = LabelEncoder()
df_signals['premature'] = lb.fit_transform(df_signals['premature'])

X_train, X_val, X_test, _, _, _ = train_val_test_split(df_clinical_information, 'premature', test_ratio=0.2,
                                                       shuffle=True, random_state=0)

  0%|          | 0/300 [00:00<?, ?it/s]

In [29]:
# Count the number of static features
if add_static_data:
    _, _, selected_columns_fit_static, _, _ = preprocess_static_data(df_static_information, X_train, X_train,
                                                                     threshold_correlation=0.85)

    num_static_features = len(selected_columns_fit_static)

if not add_static_data:
    num_static_features = 0

if model_name == 'tcn_feature_sequence':
    final_train_tcn_feature_sequence(X_train, X_val, df_signals, df_clinical_information, df_static_information,
                                     best_params, features_to_use, num_static_features=num_static_features,
                                     add_static_data=add_static_data, copies=copies)

elif model_name == 'lstm_feature_sequence':
    final_train_lstm_feature_sequence(X_train, X_val, df_signals, df_clinical_information, df_static_information,
                                      best_params, features_to_use, num_static_features=num_static_features,
                                      add_static_data=add_static_data)

Epoch [1/10] Training loss: 1.3291591326
--------------------------------------------------
Epoch [2/10] Training loss: 1.2252286832
--------------------------------------------------
Epoch [3/10] Training loss: 1.1688865344
--------------------------------------------------
Epoch [4/10] Training loss: 1.1696589470
--------------------------------------------------
Epoch [5/10] Training loss: 1.1234476149
--------------------------------------------------
Epoch [6/10] Training loss: 1.1245536367
--------------------------------------------------
Epoch [7/10] Training loss: 1.0908561746
--------------------------------------------------
Epoch [8/10] Training loss: 1.0870529135
--------------------------------------------------
Epoch [9/10] Training loss: 1.0520734807
--------------------------------------------------
Epoch [10/10] Training loss: 1.0160897632
--------------------------------------------------
TRAINING COMPLETE
Model is saved at: /Users/AFischer/Documents/PhD_onderzoek/te

## Final evaluation of the TCN and LSTM feature sequence model

**TCN sample entropy:** (with early stopping)

'tcn_data_trials_feature_sample_entropy_2022-05-12_15-11.csv' (with different weight initialization)

'2022-05-13_10-05_best_model_tcn_feature_sample_entropy_seq_final_train.pth'


**TCN peak frequency:** (with early stopping)

'tcn_data_trials_feature_peak_frequency_2022-05-12_12-09.csv' (with different weight initialization (Kaiming))

'2022-05-12_16-20_best_model_tcn_feature_peak_frequency_seq_final_train.pth'

AUC of 0.68


**TCN median frequency:** (with early stopping)


'tcn_data_trials_feature_median_frequency_2022-05-12_13-36.csv' (with differen weight initialization ((Kaiming))

'2022-05-12_16-14_best_model_tcn_feature_median_frequency_seq_final_train.pth'




**LSTM sample entropy:** (with early stopping)

'lstm_data_trials_feature_sample_entropy_2022-05-16_09-33.csv'

'2022-05-16_15-47_best_model_lstm_feature_sample_entropy_seq_final_train.pth'


**LSTM peak frequency:** (with early stopping)

'lstm_data_trials_feature_peak_frequency_2022-05-13_07-54.csv' (with different initialization)

'2022-05-13_10-27_best_model_lstm_feature_peak_frequency_seq_final_train.pth'


**LSTM median frequency:** (with early stopping)

'lstm_data_trials_feature_median_frequency_2022-05-13_10-04.csv' (with different initialization)

'2022-05-13_13-56_best_model_lstm_feature_median_frequency_seq_final_train.pth'




**Combined model with static data**

**TCN sample entropy** (with early stopping)

'tcn_data_trials_feature_sample_entropy_combined_2022-05-24_21-39.csv' (With correct initialization of final linear layer)

'tcn_data_trials_feature_sample_entropy_combined_2022-06-02_23-44.csv' (With optional model)


'2022-05-30_16-57_best_model_tcn_feature_sample_entropy_combined_seq_final_train.pth'

'2022-06-03_14-21_best_model_tcn_feature_sample_entropy_combined_seq_final_train.pth'


**TCN peak frequency:** (with early stopping)

'tcn_data_trials_feature_peak_frequency_combined_2022-05-24_14-42.csv' (With correct initialization of final linear layer)

'tcn_data_trials_feature_peak_frequency_combined_2022-06-02_15-36.csv' (With optional model)

'2022-05-25_10-24_best_model_tcn_feature_peak_frequency_combined_seq_final_train.pth'

'2022-06-03_14-10_best_model_tcn_feature_peak_frequency_combined_seq_final_train.pth'


**TCN median frequency:** (with early stopping)

'tcn_data_trials_feature_median_frequency_combined_2022-05-24_18-35.csv' (With correct initialization of final linear layer)

'tcn_data_trials_feature_median_frequency_combined_2022-06-03_08-16.csv' (With optional model)


'2022-05-30_16-54_best_model_tcn_feature_median_frequency_combined_seq_final_train.pth'

'2022-06-03_14-32_best_model_tcn_feature_median_frequency_combined_seq_final_train.pth'





**LSTM sample entropy:** (with early stopping)

'lstm_data_trials_feature_sample_entropy_combined_2022-05-25_10-51.csv' (With correct initialization of final linear layer)

'2022-05-30_17-02_best_model_lstm_feature_sample_entropy_combined_seq_final_train.pth'

'lstm_data_trials_feature_sample_entropy_combined_2022-06-03_14-40.csv' 

'2022-06-07_11-13_best_model_lstm_feature_sample_entropy_combined_seq_final_train.pth'


**LSTM peak frequency:** (with early stopping)

'lstm_data_trials_feature_peak_frequency_combined_2022-05-25_08-39.csv' (With correct initialization of final linear layer)

'lstm_data_trials_feature_peak_frequency_combined_2022-06-03_11-38.csv' (With optional model)

'2022-05-30_17-09_best_model_lstm_feature_peak_frequency_combined_seq_final_train.pth'

'2022-06-03_14-52_best_model_lstm_feature_peak_frequency_combined_seq_final_train.pth'


**LSTM median frequency:** (with early stopping)

'lstm_data_trials_feature_median_frequency_combined_2022-05-24_20-14.csv' (With correct initialization of final linear layer)

'lstm_data_trials_feature_median_frequency_combined_2022-06-03_13-30.csv'

'2022-05-30_17-12_best_model_lstm_feature_median_frequency_combined_seq_final_train.pth'

'2022-06-03_14-54_best_model_lstm_feature_median_frequency_combined_seq_final_train.pth'



In [7]:
df_signals_new = pd.read_csv(f'{data_path}/df_signals_filt.csv', sep=';')

df_clinical_information = build_clinical_information_dataframe(data_path, settings_path)
df_signals = df_signals_new.merge(df_clinical_information[[c.REC_ID_NAME, 'premature']],
                                  how='left', on=c.REC_ID_NAME)

df_demographics = build_demographics_dataframe(data_path, settings_path)
df_static_information = df_demographics.merge(df_clinical_information, how='left', on=c.REC_ID_NAME)

# Assign 0 to non-premature cases and assign 1 to premature cases
lb = LabelEncoder()
df_signals['premature'] = lb.fit_transform(df_signals['premature'])

X_train, X_val, X_test, _, _, _ = train_val_test_split(df_clinical_information, 'premature', test_ratio=0.2,
                                                       shuffle=True, random_state=0)

X_train_val = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)

  0%|          | 0/300 [00:00<?, ?it/s]

In [8]:
final_models_list = ['2022-05-13_10-05_best_model_tcn_feature_sample_entropy_seq_final_train.pth', 
                     '2022-05-12_16-20_best_model_tcn_feature_peak_frequency_seq_final_train.pth', 
                     '2022-05-12_16-14_best_model_tcn_feature_median_frequency_seq_final_train.pth',  
                     '2022-05-16_15-47_best_model_lstm_feature_sample_entropy_seq_final_train.pth', 
                     '2022-05-13_10-27_best_model_lstm_feature_peak_frequency_seq_final_train.pth', 
                     '2022-05-13_13-56_best_model_lstm_feature_median_frequency_seq_final_train.pth'
                    ]

In [20]:
final_hyper_opt_results = ['tcn_data_trials_feature_sample_entropy_2022-05-12_15-11.csv', 
                           'tcn_data_trials_feature_peak_frequency_2022-05-12_12-09.csv', 
                           'tcn_data_trials_feature_median_frequency_2022-05-12_13-36.csv',  
                           'lstm_data_trials_feature_sample_entropy_2022-05-16_09-33.csv', 
                           'lstm_data_trials_feature_peak_frequency_2022-05-13_07-54.csv', 
                           'lstm_data_trials_feature_median_frequency_2022-05-13_10-04.csv'
                          ]

In [21]:
models_type = ['tcn_feature_sequence', 'tcn_feature_sequence', 'tcn_feature_sequence', 
               'lstm_feature_sequence', 'lstm_feature_sequence', 'lstm_feature_sequence']

In [22]:
add_static_data_list = [False, False, False, False, False, False]

In [23]:
df_results = pd.DataFrame()
for (final_model_name, final_hyper_opt_results_name, model_type, static_data) in zip(final_models_list, final_hyper_opt_results, models_type, add_static_data_list):
    model_name = f'{model_type}'
    out_path_model = '/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model'
    trained_model_file_name = f'{final_model_name}'
    hyper_opt_results_file_name = f'{final_hyper_opt_results_name}'
    path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

    hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')

    # eval does not work in the case of a LSTM model with added static data,
    # you have to hard copy the best_params in that case
    best_params = eval(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])

    # best_params = {'learning_rate': 0.00835335233117339, 'layer_dim': 1, 'hidden_dim_seq': 8, 'hidden_dim_static': 23, 
    #                'weight_decay': 0.09573718148597471, 'bidirectional': False, 'num_epochs': 3, 'batch_size': 40, 
    #                'feature_name': 'sample_entropy', 'drop_out_lstm': 0.0, 'optional_model': nn.Sequential()}

    print(f'best params: {best_params}')

    add_static_data = static_data
    features_to_use = ['channel_1_filt_0.34_1_hz', 'channel_2_filt_0.34_1_hz', 'channel_3_filt_0.34_1_hz']
    
    # Count the number of static features
    if add_static_data:
        _, _, selected_columns_fit_static, _, _ = preprocess_static_data(df_static_information, X_train, X_train,
                                                                         threshold_correlation=0.85)

        num_static_features = len(selected_columns_fit_static)

    if not add_static_data:
        num_static_features = 0

    if model_name == 'tcn_feature_sequence':
        all_test_preds_samp_tcn, all_test_probs_samp_tcn, rec_ids_test_samp_tcn, results_dict = evaluate_tcn_feature_sequence(df_signals, 
                                                                                                                              df_clinical_information,
                                                                                                                              df_static_information, 
                                                                                                                              X_train_val, X_test,
                                                                                                                              trained_model_file_name, 
                                                                                                                              features_to_use, 
                                                                                                                              best_params,
                                                                                                                              add_static_data=add_static_data, 
                                                                                                                              num_static_features=num_static_features)

    elif model_name == 'lstm_feature_sequence':
        all_test_preds_peak_lstm, all_test_probs_peak_lstm, rec_ids_test_peak_lstm, results_dict = evaluate_lstm_feature_sequence(df_signals, 
                                                                                      df_clinical_information, 
                                                                                      df_static_information, 
                                                                                      X_train_val, X_test,
                                                                                      trained_model_file_name, 
                                                                                      features_to_use, 
                                                                                      best_params, 
                                                                                      add_static_data=add_static_data, 
                                                                                      num_static_features=num_static_features)

    results_dict.update({'model_file_name': trained_model_file_name, 
                         'hyper_opt_file_name': hyper_opt_results_file_name})
    
    df_results = pd.concat([df_results, pd.DataFrame.from_dict([results_dict])], axis=0)
    
df_results = df_results.reset_index(drop=True)   

best params: {'learning_rate': 0.0006109972573175081, 'num_hidden_units_per_layer': 13, 'weight_decay': 0.00765919873904176, 'kernel_size': 9, 'num_epochs': 3, 'drop_out': 0.17941476994035593, 'batch_size': 50, 'feature_name': 'sample_entropy', 'num_levels': 1, 'stride': 1}
The number of rows with infinite numbers that will be replaced with zero is: 0
The number of rows with NA numbers that will be replaced with zero is: 0
Model was saved at 3 epochs

Loading at epoch 3 saved model weights...
Precision score with mean pred: 0.5
Recall score with mean pred: 0.125
F1 score with mean pred: 0.2
AUC score with mean prob: 0.49759615384615385
AP score with mean prob: 0.18505528109483485
Precision score with max pred: 0.13559322033898305
Recall score with max pred: 1.0
F1 score with max pred: 0.23880597014925373
AUC score with max prob: 0.3221153846153846
AP score with max prob: 0.11624296803387993
Precision score with more than 1 pos pred: 0.16666666666666666
Recall score with more than 1 pos

/Users/AFischer/opt/anaconda3/envs/ai_medicine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/AFischer/opt/anaconda3/envs/ai_medicine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Model was saved at 10 epochs

Loading at epoch 10 saved model weights...
The hidden states and cells are resetted at the beginning batch 0 of test_loader
The hidden states and cells are resetted at the beginning batch 5 of test_loader
The hidden states and cells are resetted at the beginning batch 10 of test_loader
Precision score with mean pred: 0.0
Recall score with mean pred: 0.0
F1 score with mean pred: 0.0
AUC score with mean prob: 0.4591346153846154
AP score with mean prob: 0.1417221137864129
Precision score with max pred: 0.10638297872340426
Recall score with max pred: 0.625
F1 score with max pred: 0.1818181818181818
AUC score with max prob: 0.4807692307692308
AP score with max prob: 0.1946107789208187
Precision score with more than 1 pos pred: 0.125
Recall score with more than 1 pos pred: 0.75
F1 score with more than 1 pos pred: 0.21428571428571427
AUC score with mean over largest two probabilities: 0.4807692307692308
AP score with mean over largest two probabilities: 0.1995555

In [13]:
df_results

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.500000,0.125,0.200000,0.497596,0.185055,0.135593,1.000,0.238806,0.322115,0.116243,0.487981,0.167581,2022-05-13_10-05_best_model_tcn_feature_sample...,tcn_data_trials_feature_sample_entropy_2022-05...
1,0.133333,0.500,0.210526,0.540865,0.214569,0.133333,1.000,0.235294,0.685096,0.214069,0.437500,0.134173,2022-05-12_16-20_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_2022-05...
2,0.000000,0.000,0.000000,0.245192,0.096030,0.106383,0.625,0.181818,0.415865,0.131317,0.317308,0.115918,2022-05-12_16-14_best_model_tcn_feature_median...,tcn_data_trials_feature_median_frequency_2022-...
3,0.000000,0.000,0.000000,0.569712,0.156061,0.145455,1.000,0.253968,0.536058,0.151284,0.336538,0.108136,2022-05-16_15-47_best_model_lstm_feature_sampl...,lstm_data_trials_feature_sample_entropy_2022-0...
4,0.000000,0.000,0.000000,0.391827,0.116325,0.117647,0.250,0.160000,0.379808,0.116595,0.367788,0.112057,2022-05-13_10-27_best_model_lstm_feature_peak_...,lstm_data_trials_feature_peak_frequency_2022-0...
5,0.000000,0.000,0.000000,0.459135,0.141722,0.106383,0.625,0.181818,0.480769,0.194611,0.480769,0.199556,2022-05-13_13-56_best_model_lstm_feature_media...,lstm_data_trials_feature_median_frequency_2022...


In [28]:
from typing import Union
def calculate_95_ci(score: Union[float, int], sample_size: int):
    interval = 1.96 * math.sqrt((score * (1 - score)) / sample_size)

    ci = (score - interval, score + interval)

    return ci

In [105]:
# AUC max prob
print(calculate_95_ci(score=0.322115, sample_size=60))
print(calculate_95_ci(score=0.685096, sample_size=60))
print(calculate_95_ci(score=0.415865, sample_size=60))
print(calculate_95_ci(score=0.536058, sample_size=60))
print(calculate_95_ci(score=0.379808, sample_size=60))
print(calculate_95_ci(score=0.480769, sample_size=60))

(0.20387513292894535, 0.44035486707105465)
(0.5675669432407022, 0.8026250567592979)
(0.29115156361489886, 0.5405784363851012)
(0.40986996365771855, 0.6622460363422815)
(0.25700030497404636, 0.5026156950259536)
(0.35434515890429114, 0.6071928410957088)


In [106]:
# AP max prob
print(calculate_95_ci(score=0.116243, sample_size=60))
print(calculate_95_ci(score=0.214069, sample_size=60))
print(calculate_95_ci(score=0.131317, sample_size=60))
print(calculate_95_ci(score=0.151284, sample_size=60))
print(calculate_95_ci(score=0.116595, sample_size=60))
print(calculate_95_ci(score=0.194611, sample_size=60))

(0.03514123899201679, 0.1973447610079832)
(0.1102804257750448, 0.3178575742249552)
(0.04585528895200518, 0.2167787110479948)
(0.06061514310827648, 0.24195285689172352)
(0.03538671553821618, 0.19780328446178383)
(0.09443423910363523, 0.29478776089636477)


In [107]:
# AUC mean prob
print(calculate_95_ci(score=0.497596, sample_size=60))
print(calculate_95_ci(score=0.540865, sample_size=60))
print(calculate_95_ci(score=0.245192, sample_size=60))
print(calculate_95_ci(score=0.569712, sample_size=60))
print(calculate_95_ci(score=0.391827, sample_size=60))
print(calculate_95_ci(score=0.459135, sample_size=60))

(0.371080006375754, 0.624111993624246)
(0.4147708072379461, 0.666959192762054)
(0.13633613292069358, 0.3540478670793064)
(0.44443026851223616, 0.6949937314877639)
(0.26830588778863523, 0.5153481122113647)
(0.33304080723794616, 0.5852291927620539)


In [108]:
# AP mean prob
print(calculate_95_ci(score=0.185055, sample_size=60))
print(calculate_95_ci(score=0.214569, sample_size=60))
print(calculate_95_ci(score=0.096030, sample_size=60))
print(calculate_95_ci(score=0.156061, sample_size=60))
print(calculate_95_ci(score=0.116325, sample_size=60))
print(calculate_95_ci(score=0.141722, sample_size=60))

(0.08679087126023047, 0.2833191287397695)
(0.11069234553996728, 0.31844565446003276)
(0.021477721773402528, 0.1705822782265975)
(0.06423129961443064, 0.24789070038556937)
(0.03519840263606515, 0.19745159736393486)
(0.05347233353141538, 0.22997166646858458)


In [24]:
df_results

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.500000,0.125,0.200000,0.497596,0.185055,0.135593,1.000,0.238806,0.322115,0.116243,0.487981,0.167581,2022-05-13_10-05_best_model_tcn_feature_sample...,tcn_data_trials_feature_sample_entropy_2022-05...
1,0.133333,0.500,0.210526,0.540865,0.214569,0.133333,1.000,0.235294,0.685096,0.214069,0.437500,0.134173,2022-05-12_16-20_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_2022-05...
2,0.000000,0.000,0.000000,0.245192,0.096030,0.106383,0.625,0.181818,0.415865,0.131317,0.317308,0.115918,2022-05-12_16-14_best_model_tcn_feature_median...,tcn_data_trials_feature_median_frequency_2022-...
3,0.000000,0.000,0.000000,0.569712,0.156061,0.145455,1.000,0.253968,0.536058,0.151284,0.336538,0.108136,2022-05-16_15-47_best_model_lstm_feature_sampl...,lstm_data_trials_feature_sample_entropy_2022-0...
4,0.000000,0.000,0.000000,0.391827,0.116325,0.117647,0.250,0.160000,0.379808,0.116595,0.367788,0.112057,2022-05-13_10-27_best_model_lstm_feature_peak_...,lstm_data_trials_feature_peak_frequency_2022-0...
5,0.000000,0.000,0.000000,0.459135,0.141722,0.106383,0.625,0.181818,0.480769,0.194611,0.480769,0.199556,2022-05-13_13-56_best_model_lstm_feature_media...,lstm_data_trials_feature_median_frequency_2022...


In [ ]:
# We need to hard copy the hyper opt results for the seq+static models

In [40]:
hyper_opt_results_file_name = 'tcn_data_trials_feature_peak_frequency_2022-05-12_12-09.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])


{'learning_rate': 5.988662440373897e-05, 'num_hidden_units_per_layer': 7, 'weight_decay': 2.6879261626081322e-05, 'kernel_size': 9, 'num_epochs': 5, 'drop_out': 0.40601834269189535, 'batch_size': 40, 'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1}


In [43]:
best_params_tcn_comb_samp = {'learning_rate': 0.0021911688919452536, 'num_hidden_units_per_layer': 6, 
                             'weight_decay': 0.009845277753430231, 'kernel_size': 7, 'num_epochs': 10, 
                             'drop_out': 0.4770607132947341, 'batch_size': 20, 'hidden_dim_static': 15, 
                             'feature_name': 'sample_entropy', 'num_levels': 1, 'stride': 1, 
                             'optional_model': nn.Sequential(
  nn.BatchNorm1d(21, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU()
)}

In [44]:
hyper_opt_results_file_name = 'tcn_data_trials_feature_peak_frequency_combined_2022-06-02_15-36.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])

{'learning_rate': 0.0012328676965757456, 'num_hidden_units_per_layer': 5, 'weight_decay': 1.0901072682524733e-06, 'kernel_size': 7, 'num_epochs': 10, 'drop_out': 0.21606491141859585, 'batch_size': 50, 'hidden_dim_static': 23, 'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1, 'optional_model': Sequential(
  (0): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
)}


In [45]:
best_params_tcn_comb_peak = {'learning_rate': 0.0012328676965757456, 'num_hidden_units_per_layer': 5, 
                             'weight_decay': 1.0901072682524733e-06, 'kernel_size': 7, 'num_epochs': 10, 
                             'drop_out': 0.21606491141859585, 'batch_size': 50, 'hidden_dim_static': 23, 
                             'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1, 
                             'optional_model': nn.Sequential(
  nn.BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU()
)}

In [46]:
hyper_opt_results_file_name = 'tcn_data_trials_feature_median_frequency_combined_2022-06-03_08-16.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])

{'learning_rate': 0.0024069006268027846, 'num_hidden_units_per_layer': 8, 'weight_decay': 5.349125967482818e-06, 'kernel_size': 3, 'num_epochs': 10, 'drop_out': 0.4814814763802485, 'batch_size': 10, 'hidden_dim_static': 23, 'feature_name': 'median_frequency', 'num_levels': 2, 'stride': 1, 'optional_model': Sequential(
  (0): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
)}


In [47]:
best_params_tcn_comb_median = {'learning_rate': 0.0024069006268027846, 'num_hidden_units_per_layer': 8, 
                               'weight_decay': 5.349125967482818e-06, 'kernel_size': 3, 'num_epochs': 10, 
                               'drop_out': 0.4814814763802485, 'batch_size': 10, 'hidden_dim_static': 23, 
                               'feature_name': 'median_frequency', 'num_levels': 2, 'stride': 1, 
                               'optional_model': nn.Sequential(
  nn.BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU()
)}

In [48]:
hyper_opt_results_file_name = 'lstm_data_trials_feature_sample_entropy_combined_2022-06-03_14-40.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])

{'learning_rate': 0.00025779584958839915, 'layer_dim': 1, 'hidden_dim_seq': 14, 'hidden_dim_static': 22, 'weight_decay': 0.00010571973634105768, 'bidirectional': True, 'num_epochs': 9, 'batch_size': 50, 'feature_name': 'sample_entropy', 'drop_out_lstm': 0.0, 'optional_model': Sequential(
  (0): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
  (2): Linear(in_features=36, out_features=13, bias=True)
  (3): ReLU()
)}


In [49]:
best_params_lstm_comb_samp = {'learning_rate': 0.00025779584958839915, 'layer_dim': 1, 
                              'hidden_dim_seq': 14, 'hidden_dim_static': 22, 
                              'weight_decay': 0.00010571973634105768, 'bidirectional': True, 'num_epochs': 9, 
                              'batch_size': 50, 'feature_name': 'sample_entropy', 'drop_out_lstm': 0.0, 
                              'optional_model': nn.Sequential(
  nn.BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU(),
  nn.Linear(in_features=36, out_features=13, bias=True),
  nn.ReLU()
)}

In [50]:
hyper_opt_results_file_name = 'lstm_data_trials_feature_peak_frequency_combined_2022-06-03_11-38.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])


{'learning_rate': 0.008528755613393514, 'layer_dim': 1, 'hidden_dim_seq': 11, 'hidden_dim_static': 18, 'weight_decay': 0.012203652330812099, 'bidirectional': True, 'num_epochs': 5, 'batch_size': 40, 'feature_name': 'peak_frequency', 'drop_out_lstm': 0.0, 'optional_model': Sequential(
  (0): BatchNorm1d(29, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
  (2): Linear(in_features=29, out_features=18, bias=True)
  (3): ReLU()
)}


In [51]:
best_params_lstm_comb_peak = {'learning_rate': 0.008528755613393514, 'layer_dim': 1, 'hidden_dim_seq': 11, 
                              'hidden_dim_static': 18, 'weight_decay': 0.012203652330812099, 
                              'bidirectional': True, 'num_epochs': 5, 'batch_size': 40, 
                              'feature_name': 'peak_frequency', 'drop_out_lstm': 0.0, 
                              'optional_model': nn.Sequential(
  nn.BatchNorm1d(29, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU(),
  nn.Linear(in_features=29, out_features=18, bias=True),
  nn.ReLU()
)}

In [52]:
hyper_opt_results_file_name = 'lstm_data_trials_feature_median_frequency_combined_2022-06-03_13-30.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])

{'learning_rate': 5.609271537814585e-05, 'layer_dim': 2, 'hidden_dim_seq': 15, 'hidden_dim_static': 21, 'weight_decay': 0.000194408300577662, 'bidirectional': True, 'num_epochs': 7, 'batch_size': 60, 'feature_name': 'median_frequency', 'drop_out_lstm': 0.16034432192312256, 'optional_model': Sequential(
  (0): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
)}


In [53]:
best_params_lstm_comb_median = {'learning_rate': 5.609271537814585e-05, 'layer_dim': 2, 'hidden_dim_seq': 15, 
                                'hidden_dim_static': 21, 'weight_decay': 0.000194408300577662, 
                                'bidirectional': True, 'num_epochs': 7, 'batch_size': 60, 
                                'feature_name': 'median_frequency', 'drop_out_lstm': 0.16034432192312256, 
                                'optional_model': nn.Sequential(
  nn.BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  nn.ReLU()
)}

In [56]:
models_type = ['tcn_feature_sequence', 'tcn_feature_sequence', 'tcn_feature_sequence', 
               'lstm_feature_sequence', 'lstm_feature_sequence', 'lstm_feature_sequence']
add_static_data_list = [True, True, True, True, True, True]
final_models_list = ['2022-06-03_14-21_best_model_tcn_feature_sample_entropy_combined_seq_final_train.pth', 
                     '2022-06-03_14-10_best_model_tcn_feature_peak_frequency_combined_seq_final_train.pth', 
                     '2022-06-03_14-32_best_model_tcn_feature_median_frequency_combined_seq_final_train.pth', 
                     '2022-06-07_11-13_best_model_lstm_feature_sample_entropy_combined_seq_final_train.pth',
                     '2022-06-03_14-52_best_model_lstm_feature_peak_frequency_combined_seq_final_train.pth', 
                     '2022-06-03_14-54_best_model_lstm_feature_median_frequency_combined_seq_final_train.pth']
final_hyper_opt_list = ['tcn_data_trials_feature_sample_entropy_combined_2022-06-02_23-44.csv', 
                        'tcn_data_trials_feature_peak_frequency_combined_2022-06-02_15-36.csv', 
                        'tcn_data_trials_feature_median_frequency_combined_2022-06-03_08-16.csv', 
                        'lstm_data_trials_feature_sample_entropy_combined_2022-06-03_14-40.csv',
                        'lstm_data_trials_feature_peak_frequency_combined_2022-06-03_11-38.csv', 
                        'lstm_data_trials_feature_median_frequency_combined_2022-06-03_13-30.csv']
best_params_list = [best_params_tcn_comb_samp, best_params_tcn_comb_peak, best_params_tcn_comb_median, 
                    best_params_lstm_comb_samp, best_params_lstm_comb_peak, best_params_lstm_comb_median]

In [57]:
df_results = pd.DataFrame()
for (best_params, final_model_name, model_type, static_data, hyper_opt_results_file_name) in zip(best_params_list, final_models_list, models_type, add_static_data_list, final_hyper_opt_list):
    model_name = f'{model_type}'
    out_path_model = '/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model'
    trained_model_file_name = f'{final_model_name}'

    add_static_data = static_data
    features_to_use = ['channel_1_filt_0.34_1_hz', 'channel_2_filt_0.34_1_hz', 'channel_3_filt_0.34_1_hz']
    
    # Count the number of static features
    if add_static_data:
        _, _, selected_columns_fit_static, _, _ = preprocess_static_data(df_static_information, X_train, X_train,
                                                                         threshold_correlation=0.85)

        num_static_features = len(selected_columns_fit_static)

    if not add_static_data:
        num_static_features = 0

    if model_name == 'tcn_feature_sequence':
        all_test_preds_samp_tcn, all_test_probs_samp_tcn, rec_ids_test_samp_tcn, results_dict = evaluate_tcn_feature_sequence(df_signals, 
                                                                                                                              df_clinical_information,
                                                                                                                              df_static_information, 
                                                                                                                              X_train_val, X_test,
                                                                                                                              trained_model_file_name, 
                                                                                                                              features_to_use, 
                                                                                                                              best_params,
                                                                                                                              add_static_data=add_static_data, 
                                                                                                                              num_static_features=num_static_features)

    elif model_name == 'lstm_feature_sequence':
        all_test_preds_peak_lstm, all_test_probs_peak_lstm, rec_ids_test_peak_lstm, results_dict = evaluate_lstm_feature_sequence(df_signals, 
                                                                                      df_clinical_information, 
                                                                                      df_static_information, 
                                                                                      X_train_val, X_test,
                                                                                      trained_model_file_name, 
                                                                                      features_to_use, 
                                                                                      best_params, 
                                                                                      add_static_data=add_static_data, 
                                                                                      num_static_features=num_static_features)

    results_dict.update({'model_file_name': trained_model_file_name, 
                         'hyper_opt_file_name': hyper_opt_results_file_name})
    
    df_results = pd.concat([df_results, pd.DataFrame.from_dict([results_dict])], axis=0)
    
df_results = df_results.reset_index(drop=True)   

The number of rows with infinite numbers that will be replaced with zero is: 0
The number of rows with NA numbers that will be replaced with zero is: 0
Model was saved at 10 epochs

Loading at epoch 10 saved model weights...
Precision score with mean pred: 0.0625
Recall score with mean pred: 0.125
F1 score with mean pred: 0.08333333333333333
AUC score with mean prob: 0.3557692307692307
AP score with mean prob: 0.11143717393717394
Precision score with max pred: 0.041666666666666664
Recall score with max pred: 0.125
F1 score with max pred: 0.0625
AUC score with max prob: 0.34615384615384615
AP score with max prob: 0.10906511086862755
Precision score with more than 1 pos pred: 0.15151515151515152
Recall score with more than 1 pos pred: 0.625
F1 score with more than 1 pos pred: 0.24390243902439024
AUC score with mean over largest two probabilities: 0.5793269230769231
AP score with mean over largest two probabilities: 0.2516856066759874
Model was saved at 10 epochs

Loading at epoch 10 save

In [58]:
df_results

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.062500,0.125,0.083333,0.355769,0.111437,0.041667,0.125,0.062500,0.346154,0.109065,0.579327,0.251686,2022-06-03_14-21_best_model_tcn_feature_sample...,tcn_data_trials_feature_sample_entropy_combine...
1,0.000000,0.000,0.000000,0.454327,0.131540,0.115385,0.375,0.176471,0.449519,0.128757,0.576923,0.186974,2022-06-03_14-10_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_combine...
2,0.058824,0.125,0.080000,0.437500,0.127374,0.074074,0.250,0.114286,0.408654,0.120601,0.406250,0.123017,2022-06-03_14-32_best_model_tcn_feature_median...,tcn_data_trials_feature_median_frequency_combi...
3,0.090909,0.125,0.105263,0.331731,0.112448,0.100000,0.375,0.157895,0.358173,0.119672,0.579327,0.297819,2022-06-07_11-13_best_model_lstm_feature_sampl...,lstm_data_trials_feature_sample_entropy_combin...
4,0.111111,0.375,0.171429,0.382212,0.117248,0.096774,0.375,0.153846,0.391827,0.118618,0.514423,0.148127,2022-06-03_14-52_best_model_lstm_feature_peak_...,lstm_data_trials_feature_peak_frequency_combin...
5,0.142857,0.625,0.232558,0.495192,0.184714,0.139535,0.750,0.235294,0.512019,0.189106,0.502404,0.186480,2022-06-03_14-54_best_model_lstm_feature_media...,lstm_data_trials_feature_median_frequency_comb...


In [61]:
# AUC max prob
print(calculate_95_ci(score=0.346154, sample_size=60))
print(calculate_95_ci(score=0.449519, sample_size=60))
print(calculate_95_ci(score=0.408654, sample_size=60))
print(calculate_95_ci(score=0.358173, sample_size=60))
print(calculate_95_ci(score=0.391827, sample_size=60))
print(calculate_95_ci(score=0.512019, sample_size=60))

(0.2257744102250016, 0.4665335897749984)
(0.323648012460934, 0.575389987539066)
(0.2842658080856135, 0.5330421919143865)
(0.2368520339556573, 0.47949396604434275)
(0.26830588778863523, 0.5153481122113647)
(0.3855381018082452, 0.6384998981917548)


In [62]:
# AP max prob
print(calculate_95_ci(score=0.109065, sample_size=60))
print(calculate_95_ci(score=0.128757, sample_size=60))
print(calculate_95_ci(score=0.120601, sample_size=60))
print(calculate_95_ci(score=0.119672, sample_size=60))
print(calculate_95_ci(score=0.118618, sample_size=60))
print(calculate_95_ci(score=0.189106, sample_size=60))

(0.03018876755350189, 0.1879412324464981)
(0.04400781671703638, 0.21350618328296364)
(0.038196887883124184, 0.20300511211687583)
(0.037542537903329565, 0.20180146209667044)
(0.0368020771890987, 0.2004339228109013)
(0.09001935041095072, 0.2881926495890493)


In [59]:
# AUC mean prob
print(calculate_95_ci(score=0.355769, sample_size=60))
print(calculate_95_ci(score=0.454327, sample_size=60))
print(calculate_95_ci(score=0.437500, sample_size=60))
print(calculate_95_ci(score=0.331731, sample_size=60))
print(calculate_95_ci(score=0.382212, sample_size=60))
print(calculate_95_ci(score=0.495192, sample_size=60))

(0.23462963099033532, 0.4769083690096647)
(0.32833848634922086, 0.5803155136507792)
(0.3119748531169949, 0.5630251468830051)
(0.21259332163000222, 0.4508686783699978)
(0.25925526019053624, 0.5051687398094638)
(0.3686803935327634, 0.6217036064672367)


In [60]:
# AP mean prob
print(calculate_95_ci(score=0.111437, sample_size=60))
print(calculate_95_ci(score=0.131540, sample_size=60))
print(calculate_95_ci(score=0.127374, sample_size=60))
print(calculate_95_ci(score=0.112448, sample_size=60))
print(calculate_95_ci(score=0.117248, sample_size=60))
print(calculate_95_ci(score=0.184714, sample_size=60))

(0.03181386653249052, 0.19106013346750947)
(0.04601673448884995, 0.21706326551115002)
(0.04301432183375377, 0.21173367816624622)
(0.03251001120740386, 0.19238598879259616)
(0.03584272965515546, 0.19865327034484453)
(0.08651991104989343, 0.28290808895010655)


In [48]:
df_results

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.000000,0.000,0.000000,0.394231,0.119258,0.083333,0.250,0.125000,0.401442,0.119046,0.432692,0.241882,2022-05-30_16-57_best_model_tcn_feature_sample...,tcn_data_trials_feature_sample_entropy_combine...
1,0.142857,0.375,0.206897,0.449519,0.129507,0.125000,0.625,0.208333,0.466346,0.151638,0.473558,0.146380,2022-05-25_10-24_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_combine...
2,0.103448,0.375,0.162162,0.439904,0.129478,0.090909,0.375,0.146341,0.413462,0.123320,0.487981,0.166176,2022-05-30_16-54_best_model_tcn_feature_median...,tcn_data_trials_feature_median_frequency_combi...
3,0.103448,0.375,0.162162,0.447115,0.129640,0.133333,0.500,0.210526,0.449519,0.130625,0.485577,0.161304,2022-05-30_17-02_best_model_lstm_feature_sampl...,lstm_data_trials_feature_sample_entropy_combin...
4,0.111111,0.375,0.171429,0.403846,0.120195,0.111111,0.500,0.181818,0.408654,0.120509,0.538462,0.228471,2022-05-30_17-09_best_model_lstm_feature_peak_...,lstm_data_trials_feature_peak_frequency_combin...
5,0.151515,0.625,0.243902,0.439904,0.128758,0.142857,0.625,0.232558,0.454327,0.132690,0.610577,0.322517,2022-05-30_17-12_best_model_lstm_feature_media...,lstm_data_trials_feature_median_frequency_comb...


In [46]:
df_results

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.000000,0.000,0.000000,0.394231,0.119258,0.083333,0.250,0.125000,0.401442,0.119046,0.432692,0.241882,2022-05-30_16-57_best_model_tcn_feature_sample...,tcn_data_trials_feature_sample_entropy_combine...
1,0.142857,0.375,0.206897,0.449519,0.129507,0.125000,0.625,0.208333,0.466346,0.151638,0.473558,0.146380,2022-05-25_10-24_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_combine...
2,0.103448,0.375,0.162162,0.439904,0.129478,0.090909,0.375,0.146341,0.413462,0.123320,0.487981,0.166176,2022-05-30_16-54_best_model_tcn_feature_median...,tcn_data_trials_feature_median_frequency_combi...
3,0.103448,0.375,0.162162,0.447115,0.129640,0.133333,0.500,0.210526,0.449519,0.130625,0.485577,0.161304,2022-05-30_17-02_best_model_lstm_feature_sampl...,lstm_data_trials_feature_sample_entropy_combin...
4,0.111111,0.375,0.171429,0.403846,0.120195,0.111111,0.500,0.181818,0.408654,0.120509,0.538462,0.228471,2022-05-30_17-09_best_model_lstm_feature_peak_...,lstm_data_trials_feature_peak_frequency_combin...
5,0.151515,0.625,0.243902,0.439904,0.128758,0.142857,0.625,0.232558,0.454327,0.132690,0.610577,0.322517,2022-05-30_17-12_best_model_lstm_feature_media...,lstm_data_trials_feature_median_frequency_comb...


In [41]:
hyper_opt_results_file_name = 'lstm_data_trials_feature_sample_entropy_2022-05-16_09-33.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')
print(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])



{'learning_rate': 0.00011392316579449135, 'layer_dim': 2, 'hidden_dim': 8, 'weight_decay': 0.00010317203393969004, 'bidirectional': True, 'num_epochs': 3, 'batch_size': 40, 'feature_name': 'sample_entropy', 'drop_out_lstm': 0.28426873029685207}


In [82]:
model_name = 'tcn_feature_sequence'
out_path_model = '/Users/AFischer/Documents/PhD_onderzoek/term_preterm_database/output/model'
trained_model_file_name = '2022-06-03_14-10_best_model_tcn_feature_peak_frequency_combined_seq_final_train.pth'

hyper_opt_results_file_name = 'tcn_data_trials_feature_peak_frequency_combined_2022-06-02_15-36.csv'
path_to_optimal_params = f'{out_path_model}/hyper_parameter_opt/{hyper_opt_results_file_name}'

hyper_opt_params = pd.read_csv(f'{path_to_optimal_params}')

add_static_data = True
copies = False
features_to_use = ['channel_1_filt_0.34_1_hz', 'channel_2_filt_0.34_1_hz', 'channel_3_filt_0.34_1_hz']

In [83]:
hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0]

"{'learning_rate': 0.0012328676965757456, 'num_hidden_units_per_layer': 5, 'weight_decay': 1.0901072682524733e-06, 'kernel_size': 7, 'num_epochs': 10, 'drop_out': 0.21606491141859585, 'batch_size': 50, 'hidden_dim_static': 23, 'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1, 'optional_model': Sequential(\n  (0): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (1): ReLU()\n)}"

In [80]:
best_params = eval(hyper_opt_params.sort_values(by=['loss']).reset_index()['params'][0])
print(f'best params: {best_params}')

best params: {'learning_rate': 5.988662440373897e-05, 'num_hidden_units_per_layer': 7, 'weight_decay': 2.6879261626081322e-05, 'kernel_size': 9, 'num_epochs': 5, 'drop_out': 0.40601834269189535, 'batch_size': 40, 'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1}


In [84]:
# eval does not work in the case of a model with added static data,
# you have to hard copy the best_params in that case

best_params = {'learning_rate': 0.0012328676965757456, 'num_hidden_units_per_layer': 5, 
               'weight_decay': 1.0901072682524733e-06, 'kernel_size': 7, 'num_epochs': 10, 
               'drop_out': 0.21606491141859585, 'batch_size': 50, 'hidden_dim_static': 23, 
               'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1, 
               'optional_model': nn.Sequential(nn.BatchNorm1d(28, eps=1e-05, momentum=0.1, 
                                                              affine=True, track_running_stats=True), nn.ReLU())}

print(f'best params: {best_params}')

best params: {'learning_rate': 0.0012328676965757456, 'num_hidden_units_per_layer': 5, 'weight_decay': 1.0901072682524733e-06, 'kernel_size': 7, 'num_epochs': 10, 'drop_out': 0.21606491141859585, 'batch_size': 50, 'hidden_dim_static': 23, 'feature_name': 'peak_frequency', 'num_levels': 1, 'stride': 1, 'optional_model': Sequential(
  (0): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
)}


In [85]:
# Count the number of static features
if add_static_data:
    _, _, selected_columns_fit_static, _, _ = preprocess_static_data(df_static_information, X_train, X_train,
                                                                     threshold_correlation=0.85)

    num_static_features = len(selected_columns_fit_static)

if not add_static_data:
    num_static_features = 0

if model_name == 'tcn_feature_sequence':
    all_test_preds_peak_comb, all_test_probs_peak_comb, rec_ids_test, results_dict = evaluate_tcn_feature_sequence(df_signals,
                                                                                               df_clinical_information,
                                                                                               df_static_information,
                                                                                               X_train_val, X_test,
                                                                                               trained_model_file_name,
                                                                                               features_to_use,
                                                                                               best_params,
                                                                                               add_static_data=add_static_data,
                                                                                               copies=copies,
                                                                                               num_static_features=num_static_features)

elif model_name == 'lstm_feature_sequence':
    all_test_preds_samp, all_test_probs_samp, rec_ids_test, results_dict = evaluate_lstm_feature_sequence(df_signals,
                                                                                                df_clinical_information,
                                                                                                df_static_information,
                                                                                                X_train_val, X_test,
                                                                                                trained_model_file_name,
                                                                                                features_to_use,
                                                                                                best_params,
                                                                                                add_static_data=add_static_data,
                                                                                                num_static_features=num_static_features)
    
results_dict.update({'model_file_name': trained_model_file_name, 
                     'hyper_opt_file_name': hyper_opt_results_file_name})

Model was saved at 10 epochs

Loading at epoch 10 saved model weights...
Precision score with mean pred: 0.0
Recall score with mean pred: 0.0
F1 score with mean pred: 0.0
AUC score with mean prob: 0.4543269230769231
AP score with mean prob: 0.13153950758088206
Precision score with max pred: 0.11538461538461539
Recall score with max pred: 0.375
F1 score with max pred: 0.1764705882352941
AUC score with max prob: 0.4495192307692307
AP score with max prob: 0.1287565708282385
Precision score with more than 1 pos pred: 0.17647058823529413
Recall score with more than 1 pos pred: 0.75
F1 score with more than 1 pos pred: 0.2857142857142857
AUC score with mean over largest two probabilities: 0.5769230769230769
AP score with mean over largest two probabilities: 0.1869743373998237


In [86]:
df_interval_tcn_peak_comb_probs = create_interval_matrix(all_test_probs_peak_comb, rec_ids_test, 
                                                          num_sub_sequences=5)

In [87]:
df_interval_tcn_peak_comb_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.692928,0.532307,0.431865,0.454205,0.429119,1360
1,0.841453,0.468655,0.090549,0.262365,0.834652,1412
2,0.348385,0.545294,0.617057,0.154638,0.825323,1150
3,0.389104,0.565464,0.462074,0.606265,0.363342,1228
4,0.257811,0.831768,0.446586,0.459076,0.761343,604
5,0.500951,0.159507,0.753621,0.323661,0.142454,639
6,0.108276,0.455128,0.350336,0.473905,0.410402,1161
7,0.534483,0.228824,0.375925,0.348035,0.600891,966
8,0.208094,0.263773,0.090564,0.442789,0.301279,820
9,0.397227,0.527017,0.536323,0.172722,0.334094,1436


In [90]:
df_interval_tcn_peak_comb_probs[interval_columns].mean().mean()

0.44164174795150757

In [91]:
df_interval_tcn_peak_comb_probs[interval_columns].std(axis=1).mean()

0.20049697160720825

In [88]:
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs_peak, rec_ids_test, 
                                                          num_sub_sequences=5)

In [89]:
df_interval_tcn_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436


In [109]:
df_interval_tcn_peak_probs[interval_columns].mean()

interval_1    0.579044
interval_2    0.596877
interval_3    0.608462
interval_4    0.603055
interval_5    0.591864
dtype: float32

In [92]:
df_interval_tcn_peak_probs[interval_columns].mean().mean()

0.5958603024482727

In [93]:
df_interval_tcn_peak_probs[interval_columns].std(axis=1).mean()

0.09020470082759857

In [100]:
# Cases with 6 weeks or less between moment of recording and birth in test set:
rec_ids_short_ttd = [858, 1532, 553, 1007, 914]

In [101]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_short_ttd)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
17,0.495028,0.739480,0.692078,0.622346,0.564779,553
23,0.466566,0.589486,0.718441,0.659012,0.572671,1007
38,0.456896,0.612274,0.550081,0.521483,0.507764,858
41,0.476266,0.515207,0.601704,0.449855,0.510644,914
51,0.708029,0.598184,0.652287,0.631425,0.497701,1532


In [102]:
sorted_values_short_ttd = sorted(df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_short_ttd)][interval_columns].values.flat, reverse=True)

In [110]:
sorted_values_short_ttd[:7]

[0.73948026,
 0.71844125,
 0.70802885,
 0.69207764,
 0.6590124,
 0.6522874,
 0.63142514]

In [111]:
sorted_values_short_ttd[-7:]

[0.50776434,
 0.4977011,
 0.49502796,
 0.47626615,
 0.46656597,
 0.4568958,
 0.4498547]

In [104]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs.isin(sorted_values_short_ttd)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_interval_lstm_samp_comb_probs = create_interval_matrix(all_test_probs_samp_comb, rec_ids_test, 
                                                          num_sub_sequences=5)

In [77]:
df_interval_lstm_samp_comb_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.596981,0.464638,0.427737,0.423272,0.449522,1360
1,0.638736,0.276406,0.366476,0.417997,0.626469,1412
2,0.475166,0.484791,0.461781,0.196189,0.622585,1150
3,0.157804,0.317201,0.419578,0.449407,0.544225,1228
4,0.435338,0.665276,0.242380,0.381197,0.384306,604
5,0.417892,0.537021,0.595441,0.486745,0.507720,639
6,0.583305,0.506040,0.400852,0.439769,0.270678,1161
7,0.551119,0.387630,0.543660,0.402635,0.495319,966
8,0.358907,0.243658,0.201900,0.254434,0.289379,820
9,0.550307,0.471324,0.388757,0.513103,0.357246,1436


In [67]:
df_interval_lstm_samp_comb_probs[interval_columns].mean().mean()

0.43691349029541016

In [76]:
df_interval_lstm_samp_comb_probs[interval_columns].std(axis=1).mean()

0.12548469007015228

In [72]:
df_interval_lstm_samp_probs = create_interval_matrix(all_test_probs_samp, rec_ids_test, 
                                                     num_sub_sequences=5)

In [74]:
df_interval_lstm_samp_probs[interval_columns].mean().mean()

0.5017505884170532

In [75]:
df_interval_lstm_samp_probs[interval_columns].std(axis=1).mean()

0.010876879096031189

In [40]:
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs, rec_ids_test, 
                                                    num_sub_sequences=5)

In [41]:
df_interval_tcn_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436


In [34]:
df_interval_tcn_peak_probs_copies = create_interval_matrix(all_test_probs_copies, rec_ids_test, 
                                                           num_sub_sequences=5)

In [35]:
df_interval_tcn_peak_probs_copies

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.697521,0.601976,0.726387,0.485266,0.715735,1360
1,0.765780,0.158242,0.785452,0.272560,0.792573,1412
2,0.492843,0.002086,0.549045,0.001810,0.543953,1150
3,0.031299,0.681644,0.022235,0.631719,0.018361,1228
4,0.143660,0.001378,0.162894,0.005171,0.259161,604
5,0.531574,0.183455,0.553762,0.173046,0.450097,639
6,0.116114,0.339128,0.129767,0.308012,0.020078,1161
7,0.607500,0.387003,0.629409,0.251635,0.492548,966
8,0.071022,0.443780,0.047279,0.421646,0.087452,820
9,0.250359,0.064740,0.282009,0.055849,0.367252,1436


In [44]:
sorted_copies = sorted(df_interval_tcn_peak_probs_copies[interval_columns].values.flat, reverse=True)

In [45]:
top10_copies = df_interval_tcn_peak_probs_copies[df_interval_tcn_peak_probs_copies.isin(sorted_copies[:10])]

In [46]:
top10_copies.loc[top10_copies.dropna(thresh=1).index, interval_columns].join(df_interval_tcn_peak_probs_copies[['rec_id']])

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
15,NaN,0.855447,NaN,0.853676,NaN,625
21,0.814779,NaN,0.804079,NaN,NaN,1747
24,NaN,NaN,NaN,0.885658,NaN,827
45,0.848664,NaN,0.864727,NaN,0.857273,1726
51,NaN,0.810859,NaN,NaN,NaN,1532
52,NaN,0.809919,NaN,NaN,NaN,888


In [47]:
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs, 
                                                    rec_ids_test, 
                                                    num_sub_sequences=5)

In [49]:
df_interval_tcn_peak_probs[interval_columns].mean().mean()

0.5958603024482727

In [56]:
df_interval_tcn_peak_probs[interval_columns].std(axis=1).mean()

0.09020470082759857

In [51]:
df_interval_tcn_peak_probs_copies[interval_columns].mean().mean()

0.3840399384498596

In [55]:
df_interval_tcn_peak_probs_copies[interval_columns].std(axis=1).mean()

0.16693124175071716

In [118]:
results_dict

{'precision_mean_pred': 0.0,
 'recall_mean_pred': 0.0,
 'f1_mean_pred': 0.0,
 'auc_mean_prob': 0.4543269230769231,
 'ap_mean_prob': 0.13153950758088206,
 'precision_max_pred': 0.11538461538461539,
 'recall_max_pred': 0.375,
 'f1_max_pred': 0.1764705882352941,
 'auc_max_prob': 0.4495192307692307,
 'ap_max_prob': 0.1287565708282385,
 'auc_mean_largest_two_probs': 0.5769230769230769,
 'ap_mean_largest_two_probs': 0.1869743373998237,
 'model_file_name': '2022-06-03_14-10_best_model_tcn_feature_peak_frequency_combined_seq_final_train.pth',
 'hyper_opt_file_name': 'tcn_data_trials_feature_peak_frequency_combined_2022-06-02_15-36.csv'}

In [42]:
rec_ids_test_preterm = [858, 641, 1532, 1436, 1007, 914, 1302, 1476]

In [43]:
interval_columns = ['interval_1', 'interval_2', 'interval_3', 'interval_4', 'interval_5']

In [16]:
df_interval_tcn_peak_preds = create_interval_matrix(all_test_preds_peak_tcn, 
                                                         rec_ids_test_peak_tcn, 
                                                         num_sub_sequences=5)

In [17]:
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs_peak_tcn, 
                                                         rec_ids_test_peak_tcn, 
                                                         num_sub_sequences=5)

In [22]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436
23,0.466566,0.589486,0.718441,0.659012,0.572671,1007
32,0.624221,0.629253,0.580043,0.497441,0.740986,641
36,0.802446,0.711917,0.595578,0.627360,0.694641,1302
38,0.456896,0.612274,0.550081,0.521483,0.507764,858
41,0.476266,0.515207,0.601704,0.449855,0.510644,914
49,0.617336,0.536330,0.623742,0.431186,0.536806,1476
51,0.708029,0.598184,0.652287,0.631425,0.497701,1532


In [23]:
df_interval_tcn_peak_preds[df_interval_tcn_peak_preds['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
9,True,True,False,True,True,1436
23,False,True,True,True,True,1007
32,True,True,True,False,True,641
36,True,True,True,True,True,1302
38,False,True,True,True,True,858
41,False,True,True,False,True,914
49,True,True,True,False,True,1476
51,True,True,True,True,False,1532


In [24]:
df_interval_tcn_peak_probs[~df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
10,0.442749,0.525121,0.519171,0.612223,0.581096,1414


In [43]:
df_interval_tcn_peak_probs.nlargest(columns=interval_columns,n=10)

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
37,0.932795,0.555593,0.613963,0.420479,0.577226,1089
24,0.892898,0.727471,0.576765,0.660501,0.638946,827
36,0.802446,0.711917,0.595578,0.627360,0.694641,1302
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
51,0.708029,0.598184,0.652287,0.631425,0.497701,1532
46,0.695218,0.531882,0.593089,0.566682,0.556581,660
44,0.689977,0.692903,0.439848,0.569277,0.453106,891
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
31,0.666201,0.668753,0.519732,0.647687,0.553198,920
50,0.664454,0.510013,0.624188,0.556364,0.592556,716


In [53]:
test = sorted(df_interval_tcn_peak_probs[interval_columns].values.flat, reverse=True)

In [55]:
test[0:10]

[0.93279475,
 0.8928985,
 0.8840553,
 0.8378032,
 0.836039,
 0.8300072,
 0.82932794,
 0.8098591,
 0.80244553,
 0.7946337]

In [57]:
test[-10:]

[0.43726867,
 0.4361573,
 0.4311865,
 0.4217704,
 0.4204791,
 0.40223116,
 0.39051223,
 0.38013226,
 0.36987987,
 0.35153252]

In [82]:
test2 = df_interval_tcn_peak_probs[df_interval_tcn_peak_probs.isin(test[:10])]

In [88]:
test3 = df_interval_tcn_peak_probs[df_interval_tcn_peak_probs.isin(test[-10:])]

In [83]:
test2

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,0.830007,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,0.809859,NaN,NaN


In [91]:
df_interval_tcn_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436


In [94]:
df_interval_tcn_peak_probs[interval_columns].mean().mean()

0.5958603024482727

In [86]:
test2.loc[test2.dropna(thresh=1).index, interval_columns].join(df_interval_tcn_peak_probs[['rec_id']])

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
7,NaN,NaN,NaN,0.830007,NaN,966
9,NaN,NaN,NaN,0.809859,NaN,1436
12,NaN,NaN,NaN,NaN,0.837803,1630
14,NaN,NaN,NaN,NaN,0.829328,1546
24,0.892898,NaN,NaN,NaN,NaN,827
26,NaN,0.794634,NaN,NaN,NaN,1626
36,0.802446,NaN,NaN,NaN,NaN,1302
37,0.932795,NaN,NaN,NaN,NaN,1089
40,NaN,0.836039,NaN,NaN,NaN,1659
52,NaN,NaN,NaN,NaN,0.884055,888


In [89]:
test3.loc[test3.dropna(thresh=1).index, interval_columns].join(df_interval_tcn_peak_probs[['rec_id']])

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,NaN,0.437269,NaN,NaN,NaN,1360
3,NaN,NaN,NaN,NaN,0.390512,1228
34,NaN,0.351533,NaN,NaN,0.369880,697
37,NaN,NaN,NaN,0.420479,NaN,1089
42,NaN,0.380132,NaN,NaN,NaN,1492
43,0.436157,NaN,NaN,NaN,NaN,1653
47,NaN,0.421770,NaN,NaN,NaN,927
49,NaN,NaN,NaN,0.431186,NaN,1476
54,NaN,0.402231,NaN,NaN,NaN,1717


In [41]:
df_interval_tcn_peak_probs[interval_columns].stack().idxmax()

(37, 'interval_1')

In [37]:
df_interval_tcn_peak_probs[interval_columns].max(axis=1)

0     0.731413
1     0.577546
2     0.716274
3     0.780365
4     0.754331
5     0.681787
6     0.761383
7     0.830007
8     0.587057
9     0.809859
10    0.612223
11    0.723465
12    0.837803
13    0.595657
14    0.829328
15    0.643899
16    0.648694
17    0.739480
18    0.676375
19    0.659596
20    0.669505
21    0.731805
22    0.721934
23    0.718441
24    0.892898
25    0.790058
26    0.794634
27    0.672183
28    0.609503
29    0.697316
30    0.630759
31    0.668753
32    0.740986
33    0.576658
34    0.735169
35    0.672012
36    0.802446
37    0.932795
38    0.612274
39    0.642026
40    0.836039
41    0.601704
42    0.696397
43    0.700328
44    0.692903
45    0.661256
46    0.695218
47    0.647498
48    0.713661
49    0.623742
50    0.664454
51    0.708029
52    0.884055
53    0.692504
54    0.703095
55    0.717994
56    0.771361
57    0.749716
58    0.774375
59    0.764770
dtype: float32

In [26]:
interval = 1.96 * math.sqrt((0.685 * (1 - 0.685)) / 60)
print('%.3f' % interval)

0.118


In [27]:
0.685 - 0.118

0.5670000000000001

In [33]:
(0.685 - 0.118, 0.685 + 0.118)

(0.5670000000000001, 0.803)

In [18]:
results_dict

{'precision_mean_pred': 0.13333333333333333,
 'recall_mean_pred': 0.5,
 'f1_mean_pred': 0.2105263157894737,
 'auc_mean_prob': 0.5408653846153846,
 'ap_mean_prob': 0.21456942103027404,
 'precision_max_pred': 0.13333333333333333,
 'recall_max_pred': 1.0,
 'f1_max_pred': 0.23529411764705882,
 'auc_max_prob': 0.6850961538461539,
 'ap_max_prob': 0.21406931897913187,
 'auc_mean_largest_two_probs': 0.4375,
 'ap_mean_largest_two_probs': 0.1341727664351206,
 'model_file_name': '2022-05-12_16-20_best_model_tcn_feature_peak_frequency_seq_final_train.pth',
 'hyper_opt_file_name': 'tcn_data_trials_feature_peak_frequency_2022-05-12_12-09.csv'}

In [19]:
df_interval_tcn_peak_preds = create_interval_matrix(all_test_preds_peak_tcn, 
                                                    rec_ids_test_peak_tcn, 
                                                    num_sub_sequences=5)

In [20]:
df_interval_tcn_peak_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,False,True,True,True,1360
1,True,True,True,True,True,1412
2,True,True,True,True,True,1150
3,True,True,True,True,False,1228
4,True,True,True,True,True,604
5,True,True,True,True,True,639
6,True,True,True,False,True,1161
7,False,True,True,True,True,966
8,True,True,False,True,True,820
9,True,True,False,True,True,1436


In [41]:
df_interval_tcn_peak_preds[df_interval_tcn_peak_preds['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
9,True,True,False,True,True,1436
23,False,True,True,True,True,1007
32,True,True,True,False,True,641
36,True,True,True,True,True,1302
38,False,True,True,True,True,858
41,False,True,True,False,True,914
49,True,True,True,False,True,1476
51,True,True,True,True,False,1532


In [70]:
df_interval_tcn_peak_preds[~df_interval_tcn_peak_preds['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,False,True,True,True,1360
1,True,True,True,True,True,1412
2,True,True,True,True,True,1150
3,True,True,True,True,False,1228
4,True,True,True,True,True,604
5,True,True,True,True,True,639
6,True,True,True,False,True,1161
7,False,True,True,True,True,966
8,True,True,False,True,True,820
10,False,True,True,True,True,1414


In [48]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].values.mean()

0.58778065

In [49]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].values.std()

0.09433183

In [50]:
df_interval_tcn_peak_probs[~df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].values.mean()

0.5971033

In [51]:
df_interval_tcn_peak_probs[~df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].values.std()

0.096658595

In [53]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)]

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436
23,0.466566,0.589486,0.718441,0.659012,0.572671,1007
32,0.624221,0.629253,0.580043,0.497441,0.740986,641
36,0.802446,0.711917,0.595578,0.627360,0.694641,1302
38,0.456896,0.612274,0.550081,0.521483,0.507764,858
41,0.476266,0.515207,0.601704,0.449855,0.510644,914
49,0.617336,0.536330,0.623742,0.431186,0.536806,1476
51,0.708029,0.598184,0.652287,0.631425,0.497701,1532


In [68]:
df_interval_tcn_peak_probs[~df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].min(axis=1).mean()

0.4923226237297058

In [69]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].min(axis=1).mean()

0.4799771308898926

In [66]:
df_interval_tcn_peak_probs[df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].max(axis=1).mean()

0.7021850347518921

In [67]:
df_interval_tcn_peak_probs[~df_interval_tcn_peak_probs['rec_id'].isin(rec_ids_test_preterm)][interval_columns].max(axis=1).mean()

0.7151983976364136

In [21]:
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs_peak_tcn, 
                                                    rec_ids_test_peak_tcn, 
                                                    num_sub_sequences=5)

In [22]:
df_interval_tcn_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436


In [28]:
pd.DataFrame.from_dict([results_dict])

,precision_mean_pred,recall_mean_pred,f1_mean_pred,auc_mean_prob,ap_mean_prob,precision_max_pred,recall_max_pred,f1_max_pred,auc_max_prob,ap_max_prob,auc_mean_largest_two_probs,ap_mean_largest_two_probs,model_file_name,hyper_opt_file_name
0,0.133333,0.5,0.210526,0.540865,0.214569,0.133333,1.0,0.235294,0.685096,0.214069,0.4375,0.134173,2022-05-12_16-20_best_model_tcn_feature_peak_f...,tcn_data_trials_feature_peak_frequency_2022-05...


In [28]:
df_interval_tcn_samp_preds = create_interval_matrix(all_test_preds_samp_tcn, 


In [29]:
df_interval_tcn_samp_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,True,False,1360
1,True,True,True,False,True,1412
2,False,True,False,True,False,1150
3,False,True,True,False,False,1228
4,True,False,True,False,True,604
5,False,False,True,False,True,639
6,True,False,False,True,True,1161
7,False,True,False,True,True,966
8,True,False,False,False,False,820
9,True,False,True,False,False,1436


In [30]:
df_interval_tcn_samp_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.413506,0.467622,0.487071,0.574083,0.279200,1360
1,0.586303,0.626509,0.587318,0.391021,0.514556,1412
2,0.238065,0.686525,0.487457,0.530939,0.462437,1150
3,0.445460,0.612477,0.506548,0.376266,0.339174,1228
4,0.753117,0.462280,0.545440,0.448162,0.742743,604
5,0.472721,0.491653,0.625256,0.354117,0.630056,639
6,0.644666,0.238188,0.425699,0.553306,0.501085,1161
7,0.429912,0.605964,0.422547,0.786081,0.503654,966
8,0.563143,0.445083,0.424958,0.416042,0.442391,820
9,0.500048,0.479024,0.591632,0.383998,0.417759,1436


In [23]:
df_interval_tcn_peak_preds = create_interval_matrix(all_test_preds_peak_tcn, 
                                                    rec_ids_test_peak_tcn, 
                                                    num_sub_sequences=5)
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs_peak_tcn, 
                                                         rec_ids_test_peak_tcn, 
                                                         num_sub_sequences=5)

In [24]:
df_interval_tcn_peak_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,False,True,True,True,1360
1,True,True,True,True,True,1412
2,True,True,True,True,True,1150
3,True,True,True,True,False,1228
4,True,True,True,True,True,604
5,True,True,True,True,True,639
6,True,True,True,False,True,1161
7,False,True,True,True,True,966
8,True,True,False,True,True,820
9,True,True,False,True,True,1436


In [25]:
df_interval_tcn_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.731413,0.437269,0.702195,0.629149,0.639607,1360
1,0.577546,0.511114,0.525187,0.565634,0.570160,1412
2,0.546564,0.588138,0.585864,0.676564,0.716274,1150
3,0.535000,0.780365,0.682387,0.540055,0.390512,1228
4,0.602130,0.521450,0.524708,0.754331,0.669010,604
5,0.681787,0.550688,0.641626,0.641822,0.508806,639
6,0.607586,0.526957,0.761383,0.468441,0.566682,1161
7,0.493247,0.582479,0.512468,0.830007,0.571076,966
8,0.578517,0.587057,0.498091,0.562501,0.586210,820
9,0.630172,0.544691,0.444594,0.809859,0.536651,1436


In [22]:
df_interval_tcn_peak_preds_comb = create_interval_matrix(all_test_preds_peak_tcn_comb, 
                                                         rec_ids_test_peak_tcn_comb, 
                                                         num_sub_sequences=5)
df_interval_tcn_peak_probs_comb = create_interval_matrix(all_test_probs_peak_tcn_comb, 
                                                         rec_ids_test_peak_tcn_comb, 
                                                         num_sub_sequences=5)

In [17]:
df_interval_tcn_peak_preds_comb

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,False,True,True,False,1360
1,True,False,True,True,True,1412
2,False,False,True,False,True,1150
3,False,True,True,False,False,1228
4,False,False,True,False,True,604
5,True,True,True,False,True,639
6,False,True,True,False,True,1161
7,False,True,False,True,True,966
8,False,False,False,False,False,820
9,True,False,False,True,False,1436


In [18]:
df_interval_tcn_peak_probs_comb

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.540595,0.420773,0.324369,0.433251,0.403526,1360
1,0.604464,0.567762,0.374254,0.322503,0.579036,1412
2,0.585116,0.649069,0.444739,0.201531,0.620785,1150
3,0.078135,0.485956,0.512172,0.487959,0.470007,1228
4,0.461340,0.688123,0.416244,0.084237,0.613596,604
5,0.534458,0.435198,0.733607,0.435681,0.416556,639
6,0.544056,0.453204,0.559717,0.450687,0.475600,1161
7,0.593148,0.332402,0.424823,0.473440,0.522675,966
8,0.236006,0.490419,0.269673,0.449395,0.495870,820
9,0.454803,0.494780,0.387357,0.450291,0.450836,1436


In [15]:
df_interval_lstm_peak_preds = create_interval_matrix(all_test_preds_peak_lstm, rec_ids_test_peak_lstm, 
                                                     num_sub_sequences=5)
df_interval_lstm_peak_probs = create_interval_matrix(all_test_probs_peak_lstm, rec_ids_test_peak_lstm, 
                                                     num_sub_sequences=5)

In [19]:
df_interval_lstm_peak_probs

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.487490,0.502868,0.487322,0.482267,0.484371,1360
1,0.494758,0.488795,0.481737,0.485581,0.486745,1412
2,0.487784,0.484960,0.497149,0.490001,0.491677,1150
3,0.489664,0.489972,0.488497,0.480892,0.492580,1228
4,0.502465,0.493751,0.504121,0.493985,0.497200,604
5,0.498422,0.499793,0.489831,0.490442,0.493566,639
6,0.489326,0.494638,0.495205,0.485706,0.489641,1161
7,0.491825,0.499527,0.490065,0.506838,0.485414,966
8,0.495081,0.486098,0.486911,0.490395,0.482938,820
9,0.497952,0.489781,0.499187,0.481298,0.493716,1436


In [27]:
df_interval_tcn_samp_preds = create_interval_matrix(all_test_preds_samp, rec_ids_test_samp, num_sub_sequences=5)
df_interval_tcn_samp_probs = create_interval_matrix(all_test_probs_samp, rec_ids_test_samp, num_sub_sequences=5)

In [28]:
df_interval_tcn_samp

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,True,True,True,True,1360
1,True,True,True,True,True,1412
2,True,True,True,True,True,1150
3,True,True,True,True,True,1228
4,True,True,True,True,True,604
5,True,True,True,True,True,639
6,True,True,True,True,True,1161
7,True,True,True,True,True,966
8,True,True,True,True,True,820
9,True,True,True,True,True,1436


In [31]:
df_interval_tcn_peak_preds = create_interval_matrix(all_test_preds_peak_tcn, 
                                                    rec_ids_test_peak_tcn, num_sub_sequences=5)
df_interval_tcn_peak_probs = create_interval_matrix(all_test_probs_peak_tcn, rec_ids_test_peak_tcn, 
                                                    num_sub_sequences=5)

In [32]:
df_interval_tcn_peak_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,False,False,1360
1,False,False,False,False,False,1412
2,False,False,False,True,False,1150
3,False,False,False,False,False,1228
4,False,False,True,False,False,604
5,False,False,False,False,False,639
6,False,False,False,False,False,1161
7,False,True,False,False,False,966
8,False,False,False,False,False,820
9,False,False,False,False,False,1436


In [35]:
df_interval_tcn_median_preds = create_interval_matrix(all_test_preds_median_tcn, 
                                                    rec_ids_test_median_tcn, num_sub_sequences=5)
df_interval_tcn_median_probs = create_interval_matrix(all_test_probs_median_tcn, rec_ids_test_median_tcn, 
                                                    num_sub_sequences=5)

In [36]:
df_interval_tcn_median_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,True,True,True,True,True,1360
1,True,True,True,True,True,1412
2,True,True,True,True,True,1150
3,True,True,True,True,True,1228
4,True,True,True,True,True,604
5,True,True,True,True,True,639
6,True,True,True,True,True,1161
7,True,True,True,True,True,966
8,True,True,True,True,True,820
9,True,True,True,True,True,1436


In [39]:
df_interval_lstm_samp_preds = create_interval_matrix(all_test_preds_samp_lstm, 
                                                    rec_ids_test_samp_lstm, num_sub_sequences=5)
df_interval_lstm_samp_probs = create_interval_matrix(all_test_probs_samp_lstm, rec_ids_test_samp_lstm, 
                                                    num_sub_sequences=5)

In [40]:
df_interval_lstm_samp_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,False,True,1360
1,False,False,False,False,False,1412
2,False,False,False,False,False,1150
3,False,False,False,False,True,1228
4,False,False,True,False,False,604
5,False,False,False,True,False,639
6,False,False,False,False,False,1161
7,False,False,False,False,False,966
8,False,False,False,False,False,820
9,False,False,False,False,False,1436


In [44]:
df_interval_lstm_peak_preds = create_interval_matrix(all_test_preds_peak_lstm, 
                                                    rec_ids_test_peak_lstm, num_sub_sequences=5)
df_interval_lstm_peak_probs = create_interval_matrix(all_test_probs_peak_lstm, rec_ids_test_peak_lstm, 
                                                    num_sub_sequences=5)

In [45]:
df_interval_lstm_peak_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,False,False,1360
1,False,False,False,False,False,1412
2,False,False,False,False,False,1150
3,False,False,False,False,False,1228
4,False,False,False,False,False,604
5,False,False,False,False,False,639
6,False,False,False,False,False,1161
7,False,False,False,False,False,966
8,False,False,False,False,False,820
9,False,False,False,False,False,1436


In [50]:
df_interval_lstm_median_preds = create_interval_matrix(all_test_preds_median_lstm, 
                                                    rec_ids_test_median_lstm, num_sub_sequences=5)
df_interval_lstm_median_probs = create_interval_matrix(all_test_probs_median_lstm, rec_ids_test_median_lstm, 
                                                    num_sub_sequences=5)

In [51]:
df_interval_lstm_median_preds

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,False,False,1360
1,False,False,False,False,False,1412
2,False,False,False,False,False,1150
3,False,False,False,False,False,1228
4,False,False,False,False,False,604
5,False,False,False,False,False,639
6,False,False,False,False,False,1161
7,False,False,False,False,False,966
8,False,False,False,False,False,820
9,False,False,False,False,False,1436


In [17]:
df_interval_lstm_samp_preds_comb = create_interval_matrix(all_test_preds_samp_lstm_combined, 
                                                    rec_ids_test_samp_lstm_combined, num_sub_sequences=5)
df_interval_lstm_samp_probs_comb = create_interval_matrix(all_test_probs_samp_lstm_combined, 
                                                          rec_ids_test_samp_lstm_combined, 
                                                    num_sub_sequences=5)

In [18]:
df_interval_lstm_samp_preds_comb

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,False,False,False,True,True,1360
1,False,False,False,False,False,1412
2,False,False,False,True,False,1150
3,False,False,False,True,False,1228
4,False,False,False,True,False,604
5,False,False,False,False,True,639
6,False,False,False,True,False,1161
7,False,False,False,False,False,966
8,False,False,False,True,True,820
9,False,False,False,False,True,1436


In [19]:
df_interval_lstm_samp_probs_comb

,interval_1,interval_2,interval_3,interval_4,interval_5,rec_id
0,0.463039,0.487457,0.481327,0.504339,0.510018,1360
1,0.492066,0.493593,0.477674,0.489021,0.490596,1412
2,0.470212,0.484659,0.477117,0.502010,0.496083,1150
3,0.474772,0.492231,0.495236,0.508045,0.495952,1228
4,0.464100,0.480295,0.497080,0.504478,0.495394,604
5,0.458004,0.466625,0.490910,0.497796,0.506709,639
6,0.465128,0.491643,0.469520,0.500686,0.489321,1161
7,0.459562,0.493307,0.480058,0.498339,0.489683,966
8,0.460992,0.462690,0.476214,0.509920,0.526661,820
9,0.459293,0.492612,0.472591,0.499679,0.502060,1436


## Hyperparameter optimization
Carried out with the package Optuna 

Source for setting this up: https://towardsdatascience.com/hyperparameter-tuning-of-neural-networks-with-optuna-and-pytorch-22e179efc837

Comparison between hyperopt and optuna package: https://neptune.ai/blog/optuna-vs-hyperopt

We custom sort x_arr_padded and y_arr_padded such that we can correctly batchify the data using the DataLoader class.

In this function, we sort the x_arr_padded and y_arr_padded in such way that we take a batch (60 in our case) of rec_ids 
and take the first sub_seq_length (200) time steps for the first rec id, then the first sub_seq_length time steps for the second rec id, ..., the last sub_seq_length time steps of the last rec_id (of the first batch of 60 rec ids). After that, we move on to the next batch of 60 rec ids and we take the first sub_seq_length time steps of rec id 61, etc.

In the end, we want to have our data in the following form: 

There are 180 rec ids in X_train, each rec id consisting of 28800 time steps. As we want to feed the data to a stateful LSTM model, we need to ensure that we correctly batchify our data. Stateful means that the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch:

    batch1 = [sample10, sample20, sample30, sample40]
    batch2 = [sample11, sample21, sample31, sample41]
    
In the case of X_train this means that our batches will have the form as we sketched below. With $P$ being the rec id and $\textit{t}$ the timestep. In total, X_train will be batched into 432 batches, with 144 batches needed to finish an entire sequence. As we put 60 rec ids in one batch, the first 144 batches will be dedicated to the first 60 rec ids, the second 144 batches will be dedicated to rec id 61-120 and the last 144 batches will be dedicated to rec id 121-180.

The same procedure will be repeated for X_val and X_test and as they both contain only 60 rec ids, they both need 144 batches to process their rec ids.

$$B_{1} = \begin{pmatrix} P_{1}, t_{0} & P_{1}, t_{1} & P_{1}, t_{2} & ... & P_{1}, t_{198} & P_{1}, t_{199} \\ P_{2}, t_{0} & P_{2}, t_{1} & P_{2}, t_{2} & ... & P_{2}, t_{198} & P_{2}, t_{199} \\ ... & ... & ... & ... & ... & ... \\ P_{59}, t_{0} & P_{59}, t_{1} & P_{59}, t_{2} & ... & P_{59}, t_{198} & P_{59}, t_{199} \\ P_{60}, t_{0} & P_{60}, t_{1} & P_{60}, t_{2} & ... & P_{60}, t_{198} & P_{60}, t_{199} \end{pmatrix}$$

$$B_{2} = \begin{pmatrix} P_{1}, t_{200} & P_{1}, t_{201} & P_{1}, t_{202} & ... & P_{1}, t_{398} & P_{1}, t_{399} \\ P_{2}, t_{200} & P_{2}, t_{201} & P_{2}, t_{202} & ... & P_{2}, t_{398} & P_{2}, t_{399} \\ ... & ... & ... & ... & ... & ... \\  P_{59}, t_{200} & P_{59}, t_{201} & P_{59}, t_{202} & ... & P_{59}, t_{398} & P_{59}, t_{399} \\P_{60}, t_{200} & P_{60}, t_{201} & P_{60}, t_{202} & ... & P_{60}, t_{398} & P_{60}, t_{399} \end{pmatrix}$$

$$\begin{pmatrix} ... \end{pmatrix}$$

$$B_{144} = \begin{pmatrix} P_{1}, t_{28600} & P_{1}, t_{28601} & P_{1}, t_{28602} & ... & P_{1}, t_{28798} & P_{1}, t_{28799} \\ P_{2}, t_{28600} & P_{2}, t_{28601} & P_{2}, t_{28602} & ... & P_{2}, t_{28798} & P_{2}, t_{28799} \\ ... & ... & ... & ... & ... & ... \\  P_{59}, t_{28600} & P_{59}, t_{28601} & P_{59}, t_{28602} & ... & P_{59}, t_{28798} & P_{59}, t_{28799}\\P_{60}, t_{28600} & P_{60}, t_{28601} & P_{60}, t_{28602} & ... & P_{60}, t_{28798} & P_{60}, t_{28799} \end{pmatrix}$$

$$B_{145} = \begin{pmatrix} P_{61}, t_{0} & P_{61}, t_{1} & P_{61}, t_{2} & ... & P_{61}, t_{198} & P_{61}, t_{199} \\ P_{62}, t_{0} & P_{62}, t_{1} & P_{62}, t_{2} & ... & P_{62}, t_{198} & P_{62}, t_{199} \\ ... & ... & ... & ... & ... & ... \\ P_{119}, t_{0} & P_{119}, t_{1} & P_{119}, t_{2} & ... & P_{119}, t_{198} & P_{119}, t_{199} \\ P_{120}, t_{0} & P_{120}, t_{1} & P_{120}, t_{2} & ... & P_{120}, t_{198} & P_{120}, t_{199} \end{pmatrix}$$

$$\begin{pmatrix} ... \end{pmatrix}$$

$$B_{288} = \begin{pmatrix} P_{61}, t_{28600} & P_{61}, t_{28601} & P_{61}, t_{28602} & ... & P_{61}, t_{28798} & P_{61}, t_{28799} \\ P_{62}, t_{28600} & P_{62}, t_{28601} & P_{62}, t_{28602} & ... & P_{62}, t_{28798} & P_{62}, t_{28799} \\ ... & ... & ... & ... & ... & ... \\  P_{119}, t_{28600} & P_{119}, t_{28601} & P_{119}, t_{28602} & ... & P_{119}, t_{28798} & P_{119}, t_{28799}\\P_{120}, t_{28600} & P_{120}, t_{28601} & P_{120}, t_{28602} & ... & P_{120}, t_{28798} & P_{120}, t_{28799} \end{pmatrix}$$

$$B_{289} = \begin{pmatrix} P_{121}, t_{0} & P_{121}, t_{1} & P_{121}, t_{2} & ... & P_{121}, t_{198} & P_{121}, t_{199} \\ P_{122}, t_{0} & P_{122}, t_{1} & P_{122}, t_{2} & ... & P_{122}, t_{198} & P_{122}, t_{199} \\ ... & ... & ... & ... & ... & ... \\ P_{179}, t_{0} & P_{179}, t_{1} & P_{179}, t_{2} & ... & P_{179}, t_{198} & P_{179}, t_{199} \\ P_{180}, t_{0} & P_{180}, t_{1} & P_{180}, t_{2} & ... & P_{180}, t_{198} & P_{180}, t_{199} \end{pmatrix}$$

$$\begin{pmatrix} ... \end{pmatrix}$$

$$B_{432} = \begin{pmatrix} P_{121}, t_{28600} & P_{121}, t_{28601} & P_{121}, t_{28602} & ... & P_{121}, t_{28798} & P_{121}, t_{28799} \\ P_{122}, t_{28600} & P_{122}, t_{28601} & P_{122}, t_{28602} & ... & P_{122}, t_{28798} & P_{122}, t_{28799} \\ ... & ... & ... & ... & ... & ... \\  P_{179}, t_{28600} & P_{179}, t_{28601} & P_{179}, t_{28602} & ... & P_{179}, t_{28798} & P_{179}, t_{28799}\\P_{180}, t_{28600} & P_{180}, t_{28601} & P_{180}, t_{28602} & ... & P_{180}, t_{28798} & P_{180}, t_{28799} \end{pmatrix}$$